# Wikipedia Notable Life Expectancies
# [Notebook  : Data Cleaning Part 7](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean7_thanak_2022_07_26.ipynb)
### Context

The
### Objective

The
### Data Dictionary
- Feature: Description

### Importing Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### [Reading](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_clean6.db), Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean6.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean6", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 98040 rows and 43 columns.


,day,name,info,link,num_references,year,month,info_parenth,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,info_3_1,info_3_2,info_4_0,info_4_1,info_4_2,info_5_0,info_5_1,info_5_2,info_6_0,info_6_1,info_7_0,info_8_0,info_8_1,info_9_0,info_10_0,info_11_0,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,3.091042,director,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,1,0,0,0,0,0,0,1
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,68.0,None,Ireland,None,None,2.564949,None,None,and academic,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,1,0,0,1,0,0,0,2


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,info_3_1,info_3_2,info_4_0,info_4_1,info_4_2,info_5_0,info_5_1,info_5_2,info_6_0,info_6_1,info_7_0,info_8_0,info_8_1,info_9_0,info_10_0,info_11_0,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
98038,9,Aamir Liaquat Hussain,", 50, Pakistani journalist and politician, MNA .",https://en.wikipedia.org/wiki/Aamir_Liaquat_Hussain,99,2022,June,", since",50.0,None,Pakistan,None,"2002 2007, since 2018",4.605170,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,1,0,0,1,0,0,0,2
98039,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,86.0,None,"China, People's Republic of",None,None,1.386294,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,0,0,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,info_3_1,info_3_2,info_4_0,info_4_1,info_4_2,info_5_0,info_5_1,info_5_2,info_6_0,info_6_1,info_7_0,info_8_0,info_8_1,info_9_0,info_10_0,info_11_0,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
84307,23,Arthur A. Dugoni,", 95, American dentist.",https://en.wikipedia.org/wiki/Arthur_A._Dugoni,11,2020,September,None,95.0,None,United States of America,None,None,2.484907,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,0,0,0,0,0,0,0,0,0,0,0,1
63696,22,Mickey Roker,", 84, American jazz drummer, lung cancer.",https://en.wikipedia.org/wiki/Mickey_Roker,5,2017,May,None,84.0,lung cancer,United States of America,None,None,1.791759,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,1,0,0,0,0,0,0,1
65002,24,Alan Boswell,", 74, English footballer .",https://en.wikipedia.org/wiki/Alan_Boswell,8,2017,August,"Shrewsbury Town, Bolton Wanderers",74.0,None,United Kingdom of Great Britain and Northern Ireland,None,"Shrewsbury Town, Bolton Wanderers",2.197225,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,1,0,0,0,0,0,1
85262,13,Jan van Toorn,", 88, Dutch graphic designer.",https://en.wikipedia.org/wiki/Jan_van_Toorn,5,2020,November,None,88.0,None,Netherlands,None,None,1.791759,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,1,0,0,0,0,0,0,1
34014,9,Doris Burn,", 87, American children's book author and illustrator.",https://en.wikipedia.org/wiki/Doris_Burn,10,2011,March,None,87.0,None,United States of America,None,None,2.397895,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98040 entries, 0 to 98039
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   day                        98040 non-null  object 
 1   name                       98040 non-null  object 
 2   info                       98040 non-null  object 
 3   link                       98040 non-null  object 
 4   num_references             98040 non-null  int64  
 5   year                       98040 non-null  int64  
 6   month                      98040 non-null  object 
 7   info_parenth               36660 non-null  object 
 8   age                        98040 non-null  float64
 9   cause_of_death             25122 non-null  object 
 10  place_1                    97887 non-null  object 
 11  place_2                    8116 non-null   object 
 12  info_parenth_copy          36660 non-null  object 
 13  log_num_references         98040 non-null  flo

<IPython.core.display.Javascript object>

#### Observations:
- With our dataset loaded, we can pick up where we left off with extracting known_for and `cause_of_death` values.
- To avoid overwriting any existing `cause_of_death` values that might flow into 2 columns, such as, "heart and lung problems", we will update the code to combine the values.

### Extracting Remaining `known_for` and `cause_of_death` Values Continued

#### Finding `known_for` Roles and `cause_of_death` in `info_3_1`

In [6]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_1"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [7]:
# # Code to check each value
# roles_cause_list.pop()

<IPython.core.display.Javascript object>

In [8]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_1"].notna()].index
#             if "diabetes" in df.loc[index, "info_3_1"]
#             #             and df.loc[index, "politics_govt_law"] == 0
#             #             and df.loc[index, "arts"] == 1
#             #             and df.loc[index, "sciences"] == 0
#             #             and df.loc[index, "spiritual"] == 0
#             #             and df.loc[index, "law_enf_military_operator"] == 0
#             #             and df.loc[index, "sports"] == 0
#             #             and df.loc[index, "event_record_other"] == 0
#             #             and df.loc[index, "crime"] == 0
#             #             and df.loc[index, "academia_humanities"] == 0
#             #             and df.loc[index, "business_farming"] == 0
#             #                         and df.loc[index, "other_species"] == 0
#             #             and df.loc[index, "num_categories"] == 0
#         ],
#         "info_3_1",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [9]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [10]:
# # Example code to quick-check a specific entry
# df[df["info_3_1"] == "diabetes"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [11]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "an National Congress politician",
    "politician Sir Abe Bailey",
    "Conservative politician",
    "neo fascist politician",
    "provincial politician",
    "former politician",
    "politician",
    "author of the Declaration of Independence of",
    "editorial director of Antiwar com",
    "Housing",
    "housing",
    "medical education",
    "activist against animal testing",
    "activist for victims of Nazism",
    "transgender rights activist",
    "anti apartheid activist",
    "anti Communist activist",
    "lesbian rights activist",
    "environmental activist",
    "animal rights activist",
    "Black lesbian activist",
    "human rights activist",
    "civil rights activist",
    "women rights activist",
    "LGTBI rights activist",
    "Lingayatism activist",
    "health care activist",
    "LGBT rights activist",
    "gay rights activist",
    "political activist",
    "anti war activist",
    "feminist activist",
    "cannabis activist",
    "internet activist",
    "cultural activist",
    "Garifuna activist",
    "consumer activist",
    "literary activist",
    "Marxist activist",
    "social activist",
    "public activist",
    "Native activist",
    "gender activist",
    "rights activist",
    "LGBTQ activist",
    "peace activist",
    "AIDS activist",
    "diplomat",
    "outspoken critic of President Vladimir Putin",
    "government critic",
    "social critic",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "songwriter in the country music genre",
    "writer of children books",
    "funk singer songwriter",
    "science fiction writer",
    "writer of literature",
    "television writer",
    "singer songwriter",
    "screenplay writer",
    "nonfiction writer",
    "children writer",
    "fantasy writer",
    "fiction writer",
    "travel writer",
    "comics writer",
    "screenwriter",
    "scriptwriter",
    "songwriter",
    "was the author of two books on Parliament",
    "author of more than music instruction books",
    "popular science author",
    "book author",
    "children book author",
    "relationship author",
    "best selling author",
    "short story author",
    "children author",
    "cookbook author",
    "textbook author",
    "actor land National Artist",
    "big screen movie actor",
    "television actor",
    "theatre actor",
    "voice actor",
    "child actor",
    "stage actor",
    "film actor",
    "actor",
    "independent producer",
    "television producer",
    "theatrical producer",
    "R&B record producer",
    "theatre producer",
    "record producer",
    "music producer",
    "blues producer",
    "movie producer",
    "film producer",
    "BBC producer",
    "TV producer",
    "television producer",
    "video game journalist",
    "literary journalist",
    "music journalist",
    "art journalist",
    "television composer",
    "film score composer",
    "music composer",
    "samba composer",
    "film composer",
    "executive director of the Producers Guild of",
    "artistic director of the National Ballet of",
    "former deputy managing director of",
    "documentary film director",
    "pornographic director",
    "television director",
    "artistic director",
    "theatre director",
    "dubbing director",
    "theater director",
    "musical director",
    "stage director",
    "music director",
    "opera director",
    "film director",
    "wife of singing cowboy Roy Rogers",
    "singer for The Lovin' Spoonful",
    "funk singer songwriter",
    "musical comedy singer",
    "singer songwriter",
    "rockabilly singer",
    "jíbaro singer",
    "bolero singer",
    "reggae singer",
    "opera singer",
    "funk singer",
    "pop singer",
    "singer",
    "Urdu poet",
    "photography historian",
    "multi instrumental session musician",
    "musician on early Tamla sessions",
    "jazz musician",
    "soul musician",
    "R&B musician",
    "a musician",
    "musician",
    "playwright",
    "theatre activist",
    "son of novelist Ernest Hemingway",
    "novelist",
    "novels",
    "broadcaster on public media",
    "radio broadcaster",
    "broadcaster",
    "podcaster",
    "literary magazine editor",
    "chief editorialist of",
    "contributing editor",
    "editor in chief of",
    "thesaurus editor",
    "newspaper editor",
    "magazine editor",
    "first editor of",
    "graphic editor",
    "story editor",
    "editor of",
    "editor",
    "film critic for The Times",
    "literary critic",
    "theater critic",
    "theatre critic",
    "music critic",
    "film critic",
    "art critic",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "rules writer",
    "automobile journalist",
    "radio sports journalist",
    "athletic director",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "rose authority",
    "flight director in the Mission Control Center at Johnson Space Center",
    "the first director of Institute for Biological Research",
    "managing director of ONGC",
    "reprocessing",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "agricultural producer",
    "executive director of the Mori Building Company",
    "managing director of AmBank",
    "Housing Association",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "authority on",
    "director of the Curtis Institute of Music",
    "director of the Head Start Program",
    "director of Georgetown Day School",
    "director of the Planetarium",
    "museum director",
    "academic; Chancellor of the University of Dublin",
    "an academician of the Academy of Sciences",
    "academic administrator",
    "academician",
    "academic",
    "poetry teacher",
    "translator of ancient epics",
    "translator of literature",
    "comic book translator",
    "translator of",
    "translator",
    "literary historian",
    "language historian",
    "ethno historian",
    "urban historian",
    "art historian",
    "historian of medieval and",
    "historic preservationist",
    "ecclesiastical historian",
    "military historian",
    "culinary historian",
    "science historian",
    "naval historian",
    "film historian",
    "music educator",
    "art educator",
    "educationist",
    "education",
    "educator",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = []
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "prominent Latter day Saint author",
    "church music composer",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = ["philanthropist"]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = []
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "complications of pneumonia",
    "pneumonia",
    "congestive heart failure",
    "multiple organ failure",
    "chronic renal failure",
    "respiratory failure",
    "pulmonary failure",
    "cardiac failure",
    "kidney failure",
    "heart failure",
    "renal failure",
    "liver failure",
    "organ failure",
    "lung failure",
    "kidney complications",
    "kidney problems",
    "kidney ailments",
    "kidney disease",
    "kidney cancer",
    "complications from COVID",
    "COVID",
    "complications of diabetes",
    "diabetes",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [12]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_1`

In [13]:
%%time

# Column to check
column = 'info_3_1'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 25123 values in cause_of_death column.

CPU times: total: 6.22 s
Wall time: 6.21 s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [14]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84981
2    12536
3      510
0        8
4        5
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_1`

In [15]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_1"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [16]:
# # Code to check each value
# roles_cause_list.pop()

<IPython.core.display.Javascript object>

In [17]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_1"].notna()].index
#             if "House" in df.loc[index, "info_3_1"]
#             #             and df.loc[index, "politics_govt_law"] == 1
#             #             and df.loc[index, "arts"] == 0
#             #             and df.loc[index, "sciences"] == 0
#             #             and df.loc[index, "spiritual"] == 0
#             #             and df.loc[index, "law_enf_military_operator"] == 0
#             #             and df.loc[index, "sports"] == 1
#             #             and df.loc[index, "event_record_other"] == 0
#             #             and df.loc[index, "crime"] == 0
#             #             and df.loc[index, "academia_humanities"] == 0
#             #             and df.loc[index, "business_farming"] == 0
#             #                         and df.loc[index, "other_species"] == 0
#             #             and df.loc[index, "num_categories"] == 0
#         ],
#         "info_3_1",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [18]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [19]:
# # Example code to quick-check a specific entry
# df[df["info_3_1"] == "Crime Commissioner"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [20]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "activist",
    "lawyer",
    "colonial administrator",
    "life peer",
    "peer",
    "convicted dissident",
    "political dissident",
    "Castro dissident",
    "dissident",
    "Technology Policy",
    "Labrador MHA for Conception Bay East",
    "Labrador MHA",
    "conservationist",
    "Labrador House of Assembly member for St George",
    "Labrador House of Assembly for Gander",
    "Labrador House of Assembly",
    "Communications",
    "United Nations civil servant",
    "public servant",
    "civil servant",
    "attorney",
    "Finance",
    "environmentalist",
    "Majority Leader of the NY Senate",
    "deputy president of the Senate",
    "member of the State Senate",
    "Senator",
    "Senate",
    "senior judge of the District Court for the Central District of",
    "High Court judge",
    "th congressional districts",
    "nd congressional districts",
    "congressman",
    "feminist",
    "Trademark Office",
    "Foreign Trade",
    "Trade",
    "Budget",
    "ambassador to",
    "ambassador",
    "founder of the Saba Labour Party",
    "Labour Party donor",
    "Labour",
    "Health Service Commissioner",
    "Islamic Affairs; Member of the House of Representatives",
    "four time member of the House of Representatives",
    "Labrador House of Assembly member for St George",
    "Speaker of the Maine House of Representatives",
    "th Speaker of the House of Representatives",
    "Labrador House of Assembly for Gander",
    "member of the House of Lords",
    "Christel House International",
    "Labrador House of Assembly",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "writer",
    "author of",
    "author",
    "producer",
    "journalist",
    "composer",
    "director",
    "poet",
    "critic",
    "newspaper publisher",
    "music publisher",
    "book publisher",
    "publisher",
    "arranger of music for television",
    "music arranger",
    "arranger",
    "essayist",
    "television presenter",
    "radio presenter",
    "TV presenter",
    "presenter",
    "Nine Network television commentator",
    "arranger of music for television",
    "reality television cast member",
    "television hostess on cuisine",
    "television quiz contestant",
    "television talk show host",
    "reality television judge",
    "television personality",
    "CBS television network",
    "television presenter",
    "television announcer",
    "television executive",
    "television actress",
    "television host",
    "television",
    "photographer",
    "manager of actress Brooke Shields",
    "mother of actress Candice Bergen",
    "Oscar winning film actress",
    '"B" actress in the s',
    "television actress",
    "stage actress",
    "voice actress",
    "film actress",
    "actress",
    "multimedia fine artist",
    "record jacket artist",
    "installation artist",
    "performance artist",
    "comic strip artist",
    "recording artist",
    "make up artist",
    "digital artist",
    "plastic artist",
    "variety artist",
    "graphic artist",
    "visual artist",
    "comics artist",
    "escape artist",
    "light artist",
    "cyber artist",
    "mime artist",
    "sculptor",
    "guitarist",
    "dancer",
    "botanical illustrator",
    "book illustrator",
    "illustrator",
    "paintings of World War II",
    "painter",
    "orchestra conductor",
    "conductor",
    "Rolling Stones manager",
    "country music manager",
    "manager of Five Star",
    "cultural manager",
    "theatre manager",
    "stage manager",
    "jazz pianist",
    "pianist",
    "Oscar winning film actress",
    "conspiracy film maker",
    "underwater filmmaker",
    "film set designer",
    "sound film star",
    "film actress",
    "film censor",
    "film music",
    "film maker",
    "filmmaker",
    "film",
    "lyricist",
    "syndicated columnist",
    "columnist for the",
    "columnist",
    "administrator of the Old Globe Theatre",
    "musical administrator",
    "music administrator",
    "arts administrator",
    "acting coach",
    "vocal coach",
    "leader of comedic group The Drifters",
    "stand up comedian",
    "comedian",
    "last surviving husband",
    "Sonora Ponceña bands",
    "bandleader",
    "agent of TV entertainer Michael Barrymore",
    "comic entertainer",
    "entertainer",
    "radio personality",
    "radio presenter",
    "radio announcer",
    "radio host",
    "radio DJ",
    "radio show host",
    "art collector",
    "collector",
    "fashion model",
    "model",
    "record industry executive",
    "broadcasting executive",
    "record label executive",
    "studio executive",
    "chief executive",
    "news executive",
    "reality television judge",
    "competition judge",
    "socialite",
    "blogger",
    "choreographer",
    "performance artist",
    "cabaret performer",
    "stunt performer",
    "comic performer",
    "performer",
    "autobiographer",
    "biographer",
    "dramaturge",
    "dramatist",
    "reporter",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "martial artist",
    "t'ai chi teacher",
    "Capirossi team manager",
    "booking manager",
    "general manager",
    'last survivor of the Philadelphia Phillies\' "Whiz Kids" that won the National League championship',
    "athletics administrator",
    "games administrator",
    "former coach of golfer Tiger Woods",
    "basketball coach",
    "assistant coach",
    "football coach",
    "bandy player",
    "scout",
    "sports executive",
    "vice president of the International Olympic Committee",
    "two bronze medals at the Summer Olympics",
    "Olympic silver medallist",
    "Olympic figure skater",
    "silver medal Olympiad",
    "Olympic champion",
    "National Olympic",
    "Olympic sprinter",
    "Olympic medalist",
    "Olympic official",
    "Olympic athlete",
    "Winter Olympics",
    "Olympic gymnast",
    "Olympic fencer",
    "Olympic sailor",
    "Olympian",
    "Olympics",
    "MMA fighter",
    "speedway entrepreneur",
    "polar explorer",
    "explorer",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "Technology Beijing",
    "Technology Prize",
    "radio amateur",
    "various radio developments",
    "radio astronomy",
    "Director of the MIT Laboratory for Computer Science from to",
    "Biomedical Sciences",
    "sociologist",
    "mathematician",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "business executive",
    "small business",
    "businesswoman",
    "businessman",
    "hospital administrator",
    "business executive",
    "retail executive",
    "entrepreneur",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "historian",
    "philosophy professor",
    "philosopher",
    "professor at George Washington University",
    "constitutional law professor",
    "professor of literature",
    "professor of medicine",
    "university professor",
    "philosophy professor",
    "professor of music",
    "college professor",
    "professor",
    "scholar of medieval literature",
    "Shakespeare scholar",
    "literary scholar",
    "scholar",
    "university administrator",
    "ethnomusicologist",
    "musicologist",
    "art lecturer",
    "lecturer",
    "museum curator",
    "curator",
    "founder of the Boston Museum of Science",
    "member of the Academy of Sciences",
    "the Academy of Social Sciences",
    "The World Academy of Sciences",
    "theatrical pedagogue",
    "pedagogue",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "son of a professor who became a rebel leader",
    "resistance fighter during World War II",
    "guerrilla fighter",
    "freedom fighter",
    "fighter ace",
    "architect of the Air Force space",
    "Crime Commissioner",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = ["islamic theologist", "theologian"]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "charitable organization executive",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = ["convicted murderer", "murderer"]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = ["concentration camp survivor", "Holocaust survivor"]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = ["sire"]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "prostate cancer",
    "influenza following skin cancer",
    "complications from cancer",
    "bone marrow cancer",
    "colorectal cancer",
    "lymph node cancer",
    "pancreatic cancer",
    "peritoneal cancer",
    "prostate cancer",
    "tracheal cancer",
    "bladder cancer",
    "stomach cancer",
    "throat cancer",
    "breast cancer",
    "spinal cancer",
    "liver cancer",
    "colon cancer",
    "brain cancer",
    "spine cancer",
    "bowel cancer",
    "lung cancer",
    "bone cancer",
    "neck cancer",
    "cancer",
    "chronic obstructive pulmonary disease",
    "complications from Alzheimer disease",
    "complications of Alzheimer disease",
    "coronary artery disease",
    "cardiovascular disease",
    "valvular heart disease",
    "pulmonary diseases",
    "Parkinson disease",
    "Alzheimer disease",
    "heart disease",
    "liver disease",
    "lung disease",
    "complications from a stroke",
    "multiple strokes",
    "a stroke",
    "stroke",
    "emphysema",
    "Possible heart attack",
    "valvular heart disease",
    "heart complications",
    "heart condition",
    "heart problems",
    "heart disease",
    "heart attack",
    "heart failur",
    "Lewy body dementia",
    "dementia",
    "complications of multiple sclerosis",
    "amyotrophic lateral sclerosis",
    "multiple sclerosis",
    "barbiturate overdose",
    "accidental overdose",
    "morphine overdose",
    "alcohol overdose",
    "heroin overdose",
    "drug overdose",
    "cardiac complications",
    "cardiac ailment",
    "cardiac arrest",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [21]:
# Dropping info_3_1 value for entries with duplicate category value
df.loc[[index for index in df[df["info_3_1"] == "manager"].index], "info_3_1"] = ""

# Dropping info_3_1 value for entries with duplicate category value
df.loc[[index for index in df[df["info_3_1"] == "administrator"].index], "info_3_1"] = ""

# Dropping info_3_1 value for entries with duplicate category value
df.loc[[index for index in df[df["info_3_1"] == "Labrador"].index], "info_3_1"] = ""


<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [22]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_1`

In [23]:
%%time

# Column to check
column = 'info_3_1'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 25129 values in cause_of_death column.

CPU times: total: 9.03 s
Wall time: 9.04 s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [24]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84761
2    12577
3      688
0        8
4        6
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_1`

In [25]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_1"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [26]:
# # Code to check each value
# roles_cause_list.pop()

<IPython.core.display.Javascript object>

In [27]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_1"].notna()].index
#             if "osis" in df.loc[index, "info_3_1"]
#             #             and df.loc[index, "politics_govt_law"] == 0
#             #             and df.loc[index, "arts"] == 1
#             #             and df.loc[index, "sciences"] == 0
#             #             and df.loc[index, "spiritual"] == 0
#             #             and df.loc[index, "law_enf_military_operator"] == 0
#             #             and df.loc[index, "sports"] == 1
#             #             and df.loc[index, "event_record_other"] == 0
#             #             and df.loc[index, "crime"] == 0
#             #             and df.loc[index, "academia_humanities"] == 0
#             #             and df.loc[index, "business_farming"] == 0
#             #                         and df.loc[index, "other_species"] == 0
#             #             and df.loc[index, "num_categories"] == 0
#         ],
#         "info_3_1",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [28]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [29]:
# # Example code to quick-check a specific entry
# df[df["info_3_1"] == "all"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [30]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "social commentator",
    "judge",
    "barrister",
    "Chiswick",
    "Industry",
    "railroad worker",
    "Kennedy Administration official",
    "elected official in Toledo",
    "government official",
    "public official",
    "CCPPC official",
    "behavioral economist",
    "political economist",
    "economist",
    "political scientist",
    "Natural Resources",
    "anti communist zealot who helped set the stage for McCarthyism",
    "anti communist",
    "communist",
    "three time president of the senate",
    "senator",
    "Tirunelveli",
    "Social Security",
    "Nicobar Islands",
    "Caicos Islands",
    "the Islands",
    "conspiracy theorist",
    "social theorist",
    "Forestry",
    "th districts",
    "Cultural Affairs",
    "Kennedy Administration official",
    "Drug Administration reviewer",
    "Administration Commission",
    "Drug Administration",
    "political prisoner",
    "Tourism",
    "Associate Justice of the Supreme Court of the",
    "Justice of the Michigan Supreme Court",
    "Justice",
    "Energy",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "artist",
    "printmaker",
    "dog trainer",
    "public speaker",
    "memoirist",
    "a promoter of culture",
    "hip hop promoter",
    "publicist",
    "humorist",
    "cabaret impresario",
    "impresario",
    "architect",
    "vocalist for Robin Trower",
    "vocalist",
    "Emmy winning documentarian",
    "documentarian",
    "talent agent",
    "theater production designer",
    "pinball machine designer",
    "album cover designer",
    "typeface designer",
    "interior designer",
    "fashion designer",
    "costume designer",
    "garden designer",
    "stage designer",
    "font designer",
    "set designer",
    "Roll Hall of Fame",
    "foreign correspondent",
    "correspondent",
    "stuntman",
    "recorded with Linda Ronstadt",
    "multitrack tape recording",
    "recording studio owner",
    "record label founder",
    "record label owner",
    "matchbox",
    "chairman of San Diego Comic Con International",
    "reality TV personality",
    "Internet personality",
    "theatre personality",
    "TV personality",
    "flag",
    "harmonica player",
    "bukkehorn player",
    "banjo player",
    "organ player",
    "bass player",
    "concertmaster",
    "newsreader",
    "magazine cartoonist",
    "cartoonist",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "coach",
    "executive",
    "sports official",
    "professional wrestler",
    "endurance runner",
    "runner",
    "member of the Pro Football Hall of Fame",
    "a member of the Baseball Hall of Fame",
    "Sports Hall of Fame",
    "FIBA Hall of Fame",
    "holder of nine world records",
    "world record holder for",
    "world record holder",
    "world champion kickboxer",
    "boxing referee",
    "boxer",
    "former Clemson football player",
    "table tennis player",
    "rugby union player",
    "ice hockey player",
    "baseball player",
    "handball player",
    "football player",
    "cricket player",
    "bridge player",
    "polo player",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "physician",
    "psychiatrist",
    "computer scientist",
    "computer engineer",
    "behavioral physiologist",
    "micropaleontologist",
    "plant pathologist",
    "parapsychologist",
    "pharmacologist",
    "epileptologist",
    "copepodologist",
    "epistemologist",
    "meteorologist",
    "criminologist",
    "methodologist",
    "ornithologist",
    "haematologist",
    "entomologist",
    "neurologist",
    "cosmologist",
    "pathologist",
    "serologist",
    "sexologist",
    "ufologist",
    "ecologist",
    "clinical psychologist",
    "petrologist",
    "neuroscientist",
    "neurologist",
    "designer of GPS",
    "Higgs boson theorist",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "inventor of the fast food system",
    "investment banker",
    "banker",
    'Apple as the "computer for the rest of us"',
    "restaurateur",
    "farmer",
    "multi millionaire industrialist",
    "industrialist",
    "Director of the Institute of Business Administration",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "teacher",
    "inventor of Cued Speech",
    "Sciences of",
    "anthropologist",
    "anthropologist",
    "archaeologist",
    "paremiologist",
    "japanologist",
    "philologist",
    "museologist",
    "indologist",
    "sinologist",
    "evolutionary theorist",
    "intellectual",
    "linguist",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "cryptologist",
    "intelligence officer",
    "military officer",
    "police officer",
    "army officer",
    "Army officer",
    "veteran of World War II",
    "veteran of World War I",
    "World War II veteran",
    "World War I veteran",
    "oldest WWII veteran",
    "war veteran",
    "Atmospheric Administration Commissioned Officer Corps",
    "Colditz prisoner",
    "prisoner of war",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "motivational speaker",
    "gospel",
    "Anglican priest",
    "Wiccan priest",
    "Jesuit priest",
    "priestess",
    "priest",
    "archbishop of the Anglican Church of Papua New",
    "the Cayman Islands",
    "Brecon",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "humanitarian",
    "social worker",
    "project designer in the Boy Scouts of",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = [
    "convicted fraudster",
    "convicted terrorist",
    "convicted criminal",
    "convicted perjurer",
    "convicted rapist",
    "extortionist",
]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "documentary subject",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "hypertension",
    "acute myeloid leukemia",
    "leukemia",
    "stabbed",
    "run motorcycle accident",
    "run car accident",
    "run car crash",
    "run accident",
    "murdered by the Lukashenko regime",
    "murdered",
    "liver related problems",
    "respiratory problems",
    "COPD",
    "shot",
    "pancreatitis",
    "encephalitis",
    "hepatitis B",
    "hepatitis C",
    "peritonitis",
    "meningitis",
    "hepatitis",
    "sepsis",
    "lung ailments",
    "lung ailment",
    "lung transplant surgery",
    "lung infections",
    "lung condition",
    "urinary tract infection",
    "brain infection",
    "chest infection",
    "lung infections",
    "liver infection",
    "tuberculosis",
    "cirrhosis",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [31]:
# Dropping info_3_1 value for entries with duplicate category value
df.loc[[index for index in df[df["info_3_1"] == "Technology"].index], "info_3_1"] = ""

# Dropping info_3_1 value for entries with duplicate category value
df.loc[
    [
        index
        for index in df[df["info_3_1"].notna()].index
        if "Science" in df.loc[index, "info_3_1"]
    ],
    "info_3_1",
] = ""

# Dropping info_3_1 value for entries with duplicate category value
df.loc[[index for index in df[df["info_3_1"] == "the"].index], "info_3_1"] = ""

# Dropping info_3_1 value for entries with duplicate category value
df.loc[[index for index in df[df["info_3_1"] == "promoter"].index], "info_3_1"] = ""

# Dropping info_3_1 value for entries with duplicate category value
df.loc[[index for index in df[df["info_3_1"] == "s"].index], "info_3_1"] = ""

# Dropping info_3_1 value for entries with duplicate category value
df.loc[[index for index in df[df["info_3_1"] == "all"].index], "info_3_1"] = ""


<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [32]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_1`

In [33]:
%%time

# Column to check
column = 'info_3_1'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 25134 values in cause_of_death column.

CPU times: total: 7.09 s
Wall time: 7.09 s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [34]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84686
2    12606
3      734
0        8
4        6
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_1`

In [35]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_1"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [36]:
# # Code to check each value
# roles_cause_list.pop()

<IPython.core.display.Javascript object>

In [37]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_1"].notna()].index
#             if "cardinal" in df.loc[index, "info_3_1"]
#             #             and df.loc[index, "politics_govt_law"] == 0
#             #             and df.loc[index, "arts"] == 0
#             #             and df.loc[index, "sciences"] == 0
#             #             and df.loc[index, "spiritual"] == 0
#             #             and df.loc[index, "law_enf_military_operator"] == 0
#             #             and df.loc[index, "sports"] == 0
#             #             and df.loc[index, "event_record_other"] == 0
#             #             and df.loc[index, "crime"] == 0
#             #             and df.loc[index, "academia_humanities"] == 0
#             #             and df.loc[index, "business_farming"] == 0
#             #             and df.loc[index, "other_species"] == 0
#             #             and df.loc[index, "num_categories"] == 0
#         ],
#         "info_3_1",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [38]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [39]:
# # Example code to quick-check a specific entry
# df[df["info_3_1"] == "cardinal"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [40]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "political corruption was the subject of the Flood Tribunal",
    "leader of main political opposition party",
    "geopolitical energy specialist",
    "political consultant",
    "political theorist",
    "political analyst",
    "political figure",
    "political",
    "missile development",
    "development",
    "delegate to the National People Congress",
    "delegate to the Arab League",
    "three time MP",
    "MP",
    "Planning",
    "pacifist",
    "Agriculture Organization",
    "presidential primary candidate",
    "presidential candidate",
    "presidential adviser",
    "Vice Minister of the Ministry of Water Resources",
    "Executive Vice Mayor of Beijing",
    "Vice Presidential nominee",
    "Vice Minister of the Ministry of Water Resources",
    "Prime Minister of the Donetsk People Republic",
    "acting Minister of Internal Affairs of",
    "Consumer Protection Minister",
    "Minister for East Affairs",
    "Islamic Affairs Minister",
    "nine term Prime Minister",
    "Federalism Minister of a",
    "Minister of Employment",
    "Deputy Prime Minister",
    "Prime Minister of the",
    "Minister of Borders",
    "Prime Minister",
    "Health",
    "founder of Drachmi Democratic Movement Five Stars",
    "founder of the National Regeneration Movement",
    "founder of the Council for a Beautiful",
    "co founder of Jatiya Samajtantrik Dal",
    "founder of the Progress Party",
    "co founder of the Mass Party",
    "founder of Occupy Pedophilia",
    "founder of Common Cause",
    "co founder of the PSDB",
    "co founder of the Gapminder Foundation",
    "natural resources",
    "Administration",
    "minister of agriculture",
    "Agriculture Organization",
    "defector",
    "first child of civil rights leader Martin Luther King Jr",
    "leader of main political opposition party",
    "widow of fascist leader Oswald Mosley",
    "leader of the Mothers of Srebrenica",
    "leader of the Independent Party",
    "prominent Pashtun leader",
    "revolutionary leader",
    "secessionist leader",
    "civil rights leader",
    "traditional leader",
    "peasant leader",
    "tribal leader",
    "labor leader",
    "reformed Ku Klux Klan leader",
    "community leader",
    "Consumer Affairs",
    "Clerk of the Crown in Chancery",
    "Nobel Peace Prize winner",
    "Landtag of North Rhine Westphalia",
    "chairman of the Communist Party of",
    "chairman of Memorial",
    "chairman of the Ocean Affairs Council",
    "the United Nations Development Program",
    "Urban Development",
    "Rural Development",
    "Development",
    "Toronto city councillor",
    "the General Secretary of the National Union of Railwaymen",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "commentator",
    "a pioneer of electronic music",
    "early music specialist",
    "popular music",
    "music",
    "drama",
    "subject of the Oscar winning animated short",
    "animator",
    "storyteller",
    "cinematographer",
    "bibliographer",
    "mastering engineer",
    "satirist",
    "CBS Vice President",
    "game show panelist",
    "panelist",
    "narrator of the PBS series",
    "narrator",
    "co founder of the Chicago Surrealist Group",
    "co founder of The Mamas & the Papas",
    "co founder of Disneyland Records",
    "founder of Food Network",
    "co founder of magazine",
    "founder of fratricide",
    "founder of Apotex",
    "camera operator",
    "silk screen printer",
    "screenplays",
    "leader of the Philharmonia Orchestra",
    "Walk of Fame",
    "Pullitzer Prize winner",
    "Pulitzer Prize winner",
    "pageant winner",
    "Grammy winner",
    "Tony winner",
    "Prince Claus Award winner",
    "clavichordist",
    "Gracie Allen",
    "chairman of Sanlih E Television",
    "Marie Curie",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "official",
    "trainer",
    "medical delegate",
    "bodybuilder",
    "co founder of the Marathon",
    "founder of Gresini Racing",
    "Commonwealth Games gold medallist",
    "championships bronze medalist",
    "Commonwealth silver medalist",
    "bronze medallist",
    "silver medallist",
    "bronze medalist",
    "silver medalist",
    "expedition leader",
    "raced Triple Crown champion",
    "two time Boat Race winner",
    "chairman of the All Jumping Course",
    "chairman of Polo",
    "World Boxing Council light heavyweight champion",
    "WBC world light middleweight champion",
    "WBC Bantamweight Champion",
    "WBC heavyweight champion",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "inventor",
    "medical researcher",
    "conspiracy theory researcher",
    "designer",
    "co founder of the CSI",
    "founder of the NASL",
    "proponent of natural childbirth",
    "naturalist",
    "the physical properties of materials",
    "physics Nobel prize winner",
    "geophysicist",
    "physicist",
    "physics Nobel prize winner",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "hotelier",
    "co founder of INSEAD",
    "founder of Berkley",
    "co founder of Pat King of Steaks cheesesteak emporium",
    "founder of R J Corman Railroad Group",
    "co founder of Bing Lee superstores",
    "founder of the Hyundai Group",
    "co founder of Wanxiang",
    "co founder of Garmin",
    "founder of Biovail",
    "chairman of Airways from to",
    "chairman of Imasco",
    "chairman of NEC",
    "chairman of off price clothier SYMS",
    "chairman of The Leela Palaces",
    "chairman of Intel Corporation",
    "chairman of The Swatch Group",
    "chairman of Trident Seafoods",
    "chairman of Sandals Resorts",
    "chairman of Biltmore Farms",
    "chairman of Banco Safra",
    "chairman of Chen Hsong",
    "chairman of Alltech",
    "chairman of IDG",
    "Santa Fe Railway",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "literary theorist",
    "theorist",
    "lexicographer",
    "epigraphist",
    "geographer",
    "Vice Chancellor of University of Oxford",
    "founder of Project Gutenberg",
    "librarian",
    "folklorist",
    "a champion of culture",
    "head of the Tamarind Institute at the University of New",
    "Vice Chancellor of University of Oxford",
    "chancellor of Sathyabama University",
    "president of Brandeis University",
    "Rector of the Central University",
    "president of Emory University",
    "Warden of Durham University",
    "Wichita State University",
    "the University of Iowa",
    "Temple University",
    "State University",
    "Akal University",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "cryptographer",
    "Minister of National Defence",
    "Defense Minister of South",
    "Defense Minister",
    "founder of militant organization Fatah",
    "founder of the Experimental Aircraft Association",
    "founder of the Police Force",
    "leader of the Provisional Republican Army",
    "leader of Forças Populares de Abril",
    "the second highest leader of Hamas",
    "former leader of PCAA",
    "leader of Tehreek e Nafaz e Shariat e Mohammadi",
    "son of a  who became a rebel leader",
    "leader of the PFLP GC",
    "paramilitary leader",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "Galloway",
    "religious leader",
    "sect leader",
    "Catholic religious leader",
    "ecumenical leader",
    "chairman of the Bishops' Conference",
    "Lismore",
    "self help consultant",
    "self help",
    "cardinal",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "founder of Neville Fernando Teaching Hospital",
    "founder of the Bandhua Mukti Morcha",
    "commune founder",
    "a leader in the pro  movement",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = ["war criminal", "criminal", "cannibal"]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = []
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = [
    "the Preakness Stakes",
    "Blue Grass Stakes",
    "Preakness Stakes",
    "Champagne Stakes",
]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "run",
    "injuries sustained in a fall",
    "fall",
    "subsequently found dead",
    "is presumed dead",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [41]:
# Dropping info_3_1 value for entry with redundant category value
df.loc[
    df[df["link"] == "https://en.wikipedia.org/wiki/Jacques_Carelman"].index, "info_3_1"
] = ""

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [42]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_1`

In [43]:
%%time

# Column to check
column = 'info_3_1'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 25134 values in cause_of_death column.

CPU times: total: 7.05 s
Wall time: 7.06 s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [44]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84647
2    12623
3      755
0        8
4        7
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_1`

In [45]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_1"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [46]:
# # Code to check each value
# roles_cause_list.pop()

<IPython.core.display.Javascript object>

In [47]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_1"].notna()].index
#             if "mixed doubles" in df.loc[index, "info_3_1"]
#             #             and df.loc[index, "politics_govt_law"] == 0
#             #             and df.loc[index, "arts"] == 0
#             #             and df.loc[index, "sciences"] == 0
#             #             and df.loc[index, "spiritual"] == 0
#             #             and df.loc[index, "law_enf_military_operator"] == 0
#             #             and df.loc[index, "sports"] == 0
#             #             and df.loc[index, "event_record_other"] == 0
#             #             and df.loc[index, "crime"] == 0
#             #             and df.loc[index, "academia_humanities"] == 0
#             #             and df.loc[index, "business_farming"] == 0
#             #                         and df.loc[index, "other_species"] == 0
#             #             and df.loc[index, "num_categories"] == 0
#         ],
#         "info_3_1",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [48]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [49]:
# # Example code to quick-check a specific entry
# df[df["info_3_1"] == "mixed doubles"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [50]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "Vice President",
    "first president of the Republic of Northern",
    "president of the Chamber of Deputies",
    "president of the Democratic Union",
    "son of president Hafez al Assad",
    "an advisor to several presidents",
    "Gerakan Party founding president",
    "president of Johnny & Associates",
    "president of Research Council",
    "founding member of Amnesty International",
    "member of the Assembly of Experts",
    "Legislative Assembly member",
    "member of Parliament",
    "member of the CAE",
    "cabinet member",
    "military junta member",
    "lesbian policy",
    "employment",
    "conscription",
    "Pro President of City State",
    "President of Biafra",
    "Guidance Department of the Workers' Party of Korea",
    "last Socialist Party of mayor of a major city",
    "Communist Party Chief of Henan province",
    "the Earth Party",
    "Freedoms Party",
    "Life Party",
    "neo",
    "Mid Kent",
    "government co minister in the internal settlement government of",
    "minister in the Whitlam government",
    "advisor to two prime ministers",
    "twice minister of the interior",
    "twice deputy prime minister",
    "former foreign minister",
    "minister of Education",
    "deputy prime minister",
    "foreign minister of",
    "government minister",
    "cabinet minister",
    "Central Fife",
    "expert of affairs",
    "founding member of Amnesty International",
    "AM",
    "last Socialist Party of mayor of a major city",
    "former mayor of Bay City",
    "press secretary for Ronald Reagan",
    "under secretary of the Congregation for the Evangelization of Peoples",
    "secretary in charge of nuclear weapons",
    "personal secretary of Saddam Hussein",
    "conscientious objector",
    "first president of the Republic of Northern",
    "jurist",
    "Leader Of Islami Shashontantra Andolan",
    "supervisor of elections",
    "Commerce",
    "Guidance Department of the Workers' Party of Korea",
    "last Nawab of Pataudi",
    "Apostolic Nuncio to the",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "acting",
    "Frances Dee",
    "member of the Rock Steady Crew",
    "Billie Burke",
    "popularizer of the Chicken Dance",
    "Life President of the Sydney International Piano Competition",
    "draughtswoman",
    "CEO of Warner Bros Records",
    "CEO of Ruth Eckerd Hall",
    "CEO of EMI Worldwide",
    "CEO of Off White",
    "CEO of Fox News",
    "CEO of NBC",
    "Tony Orlando",
    "John Coltrane",
    "creator of Sudoku",
    "chef who worked as Michael Barry",
    "master chef",
    "chef",
    "blues saxophonist",
    "saxophonist",
    "Helene Weigel",
    "nightclub owner",
    "gallery owner",
    "club owner",
    "librettist",
    "Larbey",
    "caricaturist",
    "WABC TV anchorman",
    "news anchor",
    "Warner Bros shorts from his tenure at",
    "the Daleks",
    'David Letterman sidekick known as Larry "Bud" Melman',
    "dancing partner of Buddy Ebsen",
    'Bows"',
    "Wood Quay",
    "illustrated",
    "arts patron",
    "horticulturalist",
    "horticulturist",
    "proponent of Odissi dance",
    "dance",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "player",
    "president of the Lancashire County Cricket Club",
    "President of Cricket",
    "ARU president",
    "member of the IOC",
    "former WRU President",
    "Timberwolves",
    "owner breeder of thoroughbred race horses",
    "former owner of Baseball Astros",
    "owner of the Toronto Blue Jays",
    "owner of the Buffalo Sabres",
    "owner of San Antonio Spurs",
    "football franchise owner",
    "sports franchise owner",
    "football team owner",
    "sports team owner",
    "racehorse owner",
    "horse owner",
    "team owner",
    "Grand Prix race car driver",
    "race car driver",
    "helped found Wrestling",
    "rugby league",
    "Detroit Tigers between and",
    "cricket for",
    "cricketer",
    "Leigh",
    "cc World Championships",
    "golfer",
    "named Lake Vostok",
    "mixed doubles",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "researcher",
    "engineer",
    "president of NASSCOM",
    "discoverer of LSD",
    "Yngling class",
    "Nobel prize laureate",
    "Nobel laureate",
    "Ted Hughes",
    "Production Space Center",
    "principal investigator for STIS on the Hubble Space Telescope",
    "the Marshall Space Flight Center",
    "Life breakfast cereals",
    "moths",
    "processes",
    "the term AI",
    "the Unix operating system",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "retailer",
    "President of Jerónimo Martins",
    "CEO of MVM Group",
    "former CEO of Wegmans Food Markets",
    "CEO of Scholastic Corporation",
    "CEO of Marriott International",
    "CEO of the Ford Motor Company",
    "CEO of Pacific Lumber Company",
    "CEO of International Group",
    "CEO of the Ford Foundation",
    "CEO of Braun Corporation",
    "CEO of Drummond Company",
    "CEO of Alaska Airlines",
    "CEO of Ashland Oil Inc",
    "CEO of General Motors",
    "CEO of ZeniMax Media",
    "CEO of Merrill Lynch",
    "CEO of Bear Stearns",
    "CEO of ITC Limited",
    "CEO of Jaguar Cars",
    "CEO of Paul Stuart",
    "CEO of Roadhouse",
    "CEO of Starbucks",
    "CEO of Trafigura",
    "CEO of Cintas",
    "CEO of Qualys",
    "CEO of Amtrak",
    "CEO of Vitol",
    "CEO of Atari",
    "CEO of Sony",
    "CEO of AOL",
    "CEO of IBM",
    "pioneer of low cost airlines",
    "owner of Parma Calcio",
    "owner of Tut By",
    "Anna Nicole Smith stepson",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "preservationist",
    "president of the Union Nationale Inter universitaire",
    "president of multiple colleges",
    "the genetic classification of languages",
    "Chancellor of UC Berkeley",
    "chair of the Wordsworth Trust",
    "contributor to the Auschwitz Protocol",
    "Women Studies pioneer",
    "supporter of Esperanto",
    "archivist",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "resistance member during World War II",
    "member of the Joint Chiefs of Staff",
    "Order of recipient",
    "knight of the Military Order of William",
    "noted PoW",
    "defence minister",
    "WRAF",
    "pilot of the world smallest jet",
    "defence secretary of",
    "Aviation",
    "Cornwall Constabulary",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "two term president of the Southern Baptist Convention from to",
    "Humanistic Judaism movement",
    "Grand Master of the Sovereign Military Order of",
    "Baptist minister",
    "fundamentalist Christian minister",
    "baptist minister",
    "the Gulf",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "founding member of World Wide Fund for Nature",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = ["world second oldest living person"]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = ["Hurricane Katrina", "Preakness winner"]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "stomach aneurysm",
    "lymphoma",
    "melanoma",
    "coma",
    "anti depressants",
    "stomach aneurysm",
    "rupture of the pancreas",
    "injuries from a car accident",
    "neck injuries",
    "killed at a midnight Christmas Mass",
    "killed during the Second Intifada",
    "killed in",
    "scleroderma",
    "septic shock",
    "complications from Alzheimer",
    "drug related complications",
    "respiratory complications",
    "complications from",
    "hypothermia",
    "torture",
    "benzodiazepine intoxication",
    "alcohol intoxication",
    "drug intoxication",
    "blood clots",
    "pulmonary hemorrhage",
    "brain hemorrhage",
    "ran over",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [51]:
# Dropping info_3_1 value for entries with redundant category value
df.loc[
    [
        index
        for index in df[df["info_3_1"].notna()].index
        if "culture" in df.loc[index, "info_3_1"]
    ],
    "info_3_1",
] = ""

# Dropping info_3_1 value for entries with redundant category value
df.loc[
    [
        index
        for index in df[df["info_3_1"].notna()].index
        if "founder" in df.loc[index, "info_3_1"]
    ],
    "info_3_1",
] = ""

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [52]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_1`

In [53]:
%%time

# Column to check
column = 'info_3_1'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 25136 values in cause_of_death column.

CPU times: total: 7.06 s
Wall time: 7.07 s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [54]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84626
2    12623
3      776
0        8
4        7
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_1`

In [55]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_1"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [56]:
# # Code to check each value
# value = roles_cause_list.pop()
# value

<IPython.core.display.Javascript object>

In [57]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_1"].notna()].index
#             if value in df.loc[index, "info_3_1"]
#         ],
#         "info_3_1",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [58]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [59]:
# # Example code to quick-check a specific entry
# df[df["info_3_1"] == value]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [60]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "Golders Green",
    "Zita of Bourbon Parma",
    "Public Policy Center",
    "Chairman of Millbrook First Nation",
    "Radnorshire",
    "Pacific Affairs",
    "Press Secretary to John Fitzgerald Kennedy",
    "key Watergate figure",
    "Social Welfare",
    "eventual nuclear disarmament advocate",
    "women rights advocate",
    "women rights",
    "Clinton",
    "former Secretary of State",
    "governor general of",
    "anti prohibitionist",
    "senior counsel",
    "Commander in Chief of the",
    "Marine Resources",
    "Information",
    "human rights campaigner",
    "Isleworth",
    "Refugee Board of",
    "Rajasthan",
    "mother of Benazir Bhutto",
    "Customs",
    "Welfare",
    "Heston",
    "Castleford",
    "Inverclyde",
    "Territories",
    "Prevention",
    "Traditional Affairs",
    "Eastern Stirlingshire",
    "Fife",
    "revolutionist of the independence movement",
    "Black Rod",
    "Recreation",
    "Tipton",
    "three time governor of",
    "governor of",
    "dictator",
    "Lord Advocate",
    "Treasury Solicitor",
    "Governor General of",
    "Hebei",
    "SACP",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "showman",
    "ballet master",
    "Twitter",
    "miniseries",
    "amateur illusionist",
    "nightclub proprietor",
    "Guitar Company",
    "Bad Writing Contest",
    '"Fergus Lamont"',
    "Mayor Phineas T Bluster",
    "ethnic community advocate",
    "Blake Prize",
    "smooth countrypolitan stylist of the s",
    "Sun Records",
    "Split Enz",
    "concert violinist",
    "violinist",
    "Eddie Albert",
    "former wife of Keith Moon",
    "partner of Sir Noël Coward",
    "widow of Cyril Fletcher",
    "draughtsman",
    "maestro",
    "speaker",
    "postage stamps",
    "D expert",
    "announcer",
    "The Rolling Stones to the",
    "art connoisseur",
    "planner",
    "embroiderer",
    "Cinema",
    "special effects supervisor",
    "George Burns",
    "tenor",
    "public relations expert",
    "multi instrumentalist",
    "instrumentalist",
    "frontman of",
    "soprano",
    "father of Prince",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "international master",
    "Washington Capitals",
    "Lions",
    "VJ",
    "breeder",
    "Tour de",
    "Northern",
    "Dallas Cowboys",
    "national team",
    "Sports",
    "Sport",
    "commissioner",
    "speedway",
    "referee",
    "brother of PM Justin Trudeau",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "Telecommunications",
    "Tang drink mix",
    "doctor",
    "was at the forefront of work on Read Only Memory",
    "pioneer work on Stellar nucleosynthesis",
    "rehabilitation of disabled children",
    "hypnotist",
    "parasitology",
    "probability theory",
    "nutritionist",
    "research scientist",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "stock broker",
    "Pan Am",
    "Ovation",
    "Chairman of the Board at General Motors Corporation",
    "Chairman of the Chicago Board Options Exchange",
    "Chairman of Ford Motor Company",
    "Chairman of City National Bank",
    "Chairman of Johnson & Johnson",
    "Chairman of the Sowind Group",
    "Chairman of Target",
    "Chairman of Mobil",
    "Segway",
    "Pebble Beach",
    "CEO of the",
    "David Attenborough",
    "Chairman of Ford Motor Company",
    "financier",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "Hebrew literature",
    "Chairman of the Befaqul Madarisil Arabia",
    "first dictionary",
    "Life",
    "Director of the National Air",
    "preservation of endangered aboriginal languages",
    "the Academy of Arts",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "spymaster",
    "commander of the Selous Scouts",
    "Times war",
    "Tactics",
    "War flight nurse",
    "Purple Heart",
    "flying ace",
    "Analysis Wing",
    "Army Materiel Command",
    "recovery effort after the September attacks",
    "typed Oskar Schindler list",
    "Hero of the Federation",
    "Medal of Honor recipient",
    "WWII Air Force pilot",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "New Testament",
    'Liberation" Catholic youth movement',
    "Archdeacon of Portsmouth",
    "Fili",
    "astrologer",
    "Archbishop of Maringá",
    "Archbishop of Toronto",
    "archbishop",
    "bishop",
    "Veliki Preslav",
    "Titular Bishop of Flenucleta",
    "Chakma raja",
    "reformer",
    "Edmonton",
    "muhaddith",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = ["matron of Glasgow Victoria Infirmary"]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = ["illegal oil broker", "gangster"]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "involved in Raymond Caruana murder",
    "oldest person ever born in",
    "oldest ever citizen",
    "inspiration for the movie character shootout",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = [
    "Christmas Hurdle",
    "terrier mix",
    "mascot for the Trenton Thunder",
    "Breeders' Cup Turf",
    "broodmare",
    "Prix Vermeille",
]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "Stevens Johnson syndrome",
    "or cerebral edema",
    "pulmonary edema",
    "drowned",
    "blunt force trauma",
    "lacerations",
    "morphine",
    "coronary artery plaque",
    "Valium",
    "a broken hip",
    "oxycodone",
    "myelodysplastic syndromes",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [61]:
# Dropping info_3_1 value for entries with duplicate category value
df.loc[
    [
        index
        for index in df[df["info_3_1"].notna()].index
        if "creator" in df.loc[index, "info_3_1"]
    ],
    "info_3_1",
] = ""

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [62]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_1`

In [63]:
%%time

# Column to check
column = 'info_3_1'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 25136 values in cause_of_death column.

CPU times: total: 5.66 s
Wall time: 5.65 s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [64]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84608
2    12630
3      787
0        8
4        7
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_1`

In [65]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_1"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [66]:
# # Code to check each value
# value = roles_cause_list.pop()
# value

<IPython.core.display.Javascript object>

In [67]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_1"].notna()].index
#             if value in df.loc[index, "info_3_1"]
#         ],
#         "info_3_1",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [68]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [69]:
# # Example code to quick-check a specific entry
# df[df["info_3_1"] == value]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [70]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "Promotion of Employment",
    "Employment",
    "Sale East",
    "Levies",
    "United Nations",
    "ZANLA",
    "OPEC",
    "Chief Judge of the Supreme Court",
    "Wadiyar dynasty",
    "South Tyrone",
    "Marxist",
    "Services",
    "President of",
    "sister of Robert Mugabe",
    "MSP for Strathkelvin",
    "Middleton",
    "public health expert",
    "Hillsborough",
    "Dublin West",
    "Poplar",
    "legalisation campaigner",
    "campaigner",
    "Arthabaska",
    "Loudoun",
    "Thrace",
    "trans nations builder",
    "Chatham",
    "Exchange Commission",
    "Royton",
    "Bohus",
    "Food",
    "Natural Gas",
    "Nixon Enemy",
    "Redditch",
    "Public Service Reform",
    "NTUC",
    "civil rights litigator",
    "aristocrat",
    "zionist",
    "Minister",
    "Native rights",
    "Commercial Workers Int'l",
    "segregationist",
    "legislator",
    "animal rights advocate",
    "civil rights advocat",
    "Solicitor General of the",
    "Elena Ceaușescu",
    "courtier",
    "pretender to the throne",
    "labor organizer",
    "banking regulator",
    "State Treasurer for West Virginia",
    "economic affairs",
    "Urban Habitat",
    "gay rights campaigner",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "piper",
    "Dean",
    "other game shows",
    "mother of Daniel Day Lewis",
    "father of Nicolas Cage",
    "piano",
    "Premio Campiello recipient",
    "humourist",
    "stage practitioner",
    "CEO of",
    "muralist",
    "beauty queen",
    "clown",
    "Roses",
    "won nine Grammy Awards",
    "Cologne",
    "the Road Runner",
    "antiquities dealer",
    "orator",
    "auctioneer",
    "TV scripts",
    "radio",
    "baritone",
    "Eva Gabor",
    "stamp auctioneer",
    "advocate of the arts",
    "art supporter",
    "Sci Fi movies",
    "Charlie Parker",
    "furniture maker",
    "Apollo Theater",
    "De Facto",
    "companion of Gore Vidal",
    "drummer",
    "harpsichordist",
    "non fiction on Native s",
    '" Mogul"',
    "Ella Fitzgerald",
    "magazine covers",
    "produced Top albums",
    "six time Academy Awards nominee",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "surfer",
    "Giro d'Italia in",
    "waterskier",
    "sailmaker",
    "alpinist",
    "long jumper",
    "father of Hugh Laurie",
    "athlete",
    "javelin thrower",
    "mountaineer",
    "ski orienteer",
    "the Philadelphia Phillies",
    "Mike Tyson",
    "world champion",
    "climber",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "chemist",
    "wear fabrics",
    "assigned use of @ sign",
    "motorcycle builder",
    "paranormalist",
    "the Executive Director of the Gerontology Research Group",
    "the folding ping pong table",
    "the Nanchang J",
    "pediatrician",
    "statistics",
    "Carboniferous periods",
    "patent holder",
    'coined the term "transistor"',
    "design scientist",
    "color vision",
    "chloroplast biology",
    "quantum optics",
    "fats",
    "the likelihood principle",
    "solar plasma spectroscopy",
    "fractals",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "Esprit",
    "Goodyear",
    "billionaire",
    "Popsicles",
    "restaurant proprietor",
    "corporate raider",
    "viticulturist",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "semiotician",
    "theatre instructor",
    "instructor",
    "language",
    "a specialist in the Berber languages",
    "Heritage Center",
    "reading advocate",
    "cultural analysis",
    "Syriacist",
    "an expert on Pascal",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "Atmospheric ed Officer Corps",
    "G I Joe",
    "Chief of the Navy General Staff",
    "Border Patrol",
    "missions",
    "Defence Staff Chief",
    "mercenary",
    "Army Ranger",
    "linked to al Qaeda",
    "war hero",
    "aquanaut",
    "CIA operative",
    "Al Itihaad al Islamiya",
    "test pilot",
    "militant for the independence of",
    "Federal Bureau of Investigation informer",
    "World War II OSS operative",
    "Management at the Department of Defense",
    "a confessed spy for the KGB",
    "Corregidor",
    "last known person to see Anne Franke",
    "World War II war",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "Furness",
    "Malankara Metropolitan",
    "The Isles",
    "All",
    "Eastern",
    "Ossory",
    "Biblical revisionist",
    "Byblos",
    "Bishop of Nitra",
    "creationist",
    "father of Christian Reconstructionism",
    "Islamic",
    "rabbi",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = ["the Odebrecht Foundation", "Chicago", "Family Centres"]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = [
    "loan crisis",
    "firearms possession",
    "crimes against humanity during the Civil War",
    "rapist",
]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "supercentenarian",
    "renounced the office of",
    "father of Balthazar Getty",
    "envoy to the Republic",
    "yoga adept",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = ["Racehorse of the Year"]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "exhaustion",
    "hoof inflammation",
    "helmet to shield others from a grenade explosion",
    "methyl alcohol poisoning",
    "acute respiratory distress syndrome",
    "craniocerebral trauma",
    "asthma",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [71]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_1`

In [72]:
%%time

# Column to check
column = 'info_3_1'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 25136 values in cause_of_death column.

CPU times: total: 5.83 s
Wall time: 5.82 s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [73]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84581
2    12635
3      808
0        8
4        8
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_1`

In [74]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_1"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [75]:
# # Code to check each value
# value = roles_cause_list.pop()
# value

<IPython.core.display.Javascript object>

In [76]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_1"].notna()].index
#             if value in df.loc[index, "info_3_1"]
#         ],
#         "info_3_1",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [77]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [78]:
# # Example code to quick-check a specific entry
# df[df["info_3_1"] == value]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [79]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "Fine Arts",
    "MLA for Haidergarh",
    "Skegness",
    "lobbyist",
    "Home Affairs",
    "nd Districts",
    "Receiver General of Massachusetts",
    "New",
    "Industrial Research",
    "International Security Affairs",
    "Wisconsin State Assembly",
    "government advisor",
    "Korea",
    "Mines",
    "Legal Affairs Commission",
    "IAWG",
    "Nicobar Island",
    "Patagonia",
    "Livestock",
    "Nevis",
    "counsel",
    "Wallington",
    "Family Affairs",
    "Camborne",
    "Constitutional Convention",
    "Industrial Research",
    "Research",
    "Aged Care",
    "Spenborough",
    "Spen",
    "Arab tolerance",
    "East Perthshire",
    "social insurance",
    "information",
    "sportsman",
    "sports",
    "Bihar MLC",
    "shipping",
    "energy",
    "mineral resources",
    "treasury solicitor",
    "emigrants",
    "Social Council",
    "human services",
    "public affairs",
    "industry",
    "family welfare",
    "monopolies",
    "co prince of",
    "treasurer of Queensland",
    "the International Center on Nonviolent Conflict",
    "whistleblower",
    "three term TD",
    "environment protection",
    "urbanism",
    "Sarawak",
    "tourism",
    "science",
    "foreign affairs of the PDNC",
    "social security",
    "plenipotentiary of",
    "signatory of the Act of the Re Establishment of the State of",
    "commerce",
    "Chippewa Indians",
    "literacy",
    "coiner of country name",
    "Haryana High Court",
    "Mass Education",
    "Drugs",
    "Acting Surgeon General of the",
    "Democracy",
    "Co Prince of",
    "Leader of the Opposition",
    "chief ideologist of Khmer Rouge",
    "Treasurer of UKIP",
    "Social Affairs",
    "Deputy Speaker of Parliament",
    "animal rights",
    "Political Rights Association",
    "Transportation",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "mixer",
    "organist",
    "flautist",
    "society figure",
    "gossip celebrity",
    "trombonist",
    "personal assistant to",
    "Colossus Records",
    "Imagineer",
    "cook",
    "gardener",
    "talk show host",
    "clarinetist",
    "book binder",
    "talkback host",
    "accordionist",
    "televangelist",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "sport fisherman",
    "Commonwealth Games champion",
    "sportsman",
    "larger than life persona",
    "Thoroughbred racehorse",
    "Colorado Avalanche",
    "football",
    "World Hockey Association",
    "Rushden & Diamonds F C",
    "jetpack pilot",
    "Atlanta Braves",
    "racehorse",
    "Open women doubles champion",
    "the Washington Wizards",
    "lawn bowler",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "asteroids",
    "the Arctic",
    "soil science",
    "HQ Trivia",
    "LBO",
    "the Draper Prize",
    "pharmacist",
    "builder",
    "electron spin resonance spectroscopy",
    "organ transplantation",
    "chemotherapy",
    "medical scientist",
    "scientist",
    "discoverer of REM sleep",
    "statistician",
    "medical scientist",
    "medical",
    "Academy of Engineering",
    "Yaogan satellites",
    "first flight simulator",
    "Computational Chemists",
    "developer of the image scanner",
    "engine",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "HNA Group",
    "Semapa",
    "the CITIC Group",
    "marketer",
    "Yulon",
    "Roland Corporation",
    "grazier",
    "SGV & Company",
    "real estate developer",
    "chairman of Primark",
    "EasyJet",
    "Sun International",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "design at the Museum of Modern Art",
    "Education Center",
    "CQUT",
    "orientalist",
    "celebrated the evolution of type",
    "Albert Museum",
    "president",
    "Tape Archive",
    "archdruid",
    "dean of the Padma Seshadri Bala Bhavan",
    "Design",
    "Surgeons",
    "study",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "Artillery",
    "recovery following",
    "adviser to the National Security Council",
    "Springhill mining disasters",
    "CICPC investigator",
    "PIRA",
    "the No Fighter Squadrons",
    "spy",
    "paramilitary",
    "aviator",
    "Guangzhou Military Region Air Forces",
    "Marshal of the Union",
    "tank commander",
    "Security Service",
    "Air Defence Forces Army",
    "Artillery",
    "chief of Haro lockdown",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "theistic evolutionist",
    "Edinburgh",
    "Wells",
    "Mashonaland",
    "pagan revivalist",
    "Jews",
    "Kilmacduagh",
    "Pihopa o Aotearoa",
    "Opus Dei",
    "preacher",
    "esotericist",
    "Forzol",
    "Fort de",
    "Brighton",
    "Herzegovina",
    "Lorraine",
    "Slutsk",
    "Prizren",
    "Chukotka",
    "St Vincent",
    "Kuruman",
    "Raphoe",
    "Minneapolis",
    "Ipswich",
    "Suzdal",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = ["informant", "human rights violator", "embezzlement", "kidnapper"]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "last living verified person born in the th century",
    "kidnap victim",
    "parental rights case",
    "imprisoned by",
    "murder",
    "the Siege of Sarajevo",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = ["Ascot Gold Cup", "Derby winner", "hatched in captivity"]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "ALS",
    "esophageal dysphagia",
    "swine flu",
    "brain trauma",
    "respiratory stress syndrome",
    "chronic traumatic encephalopathy",
    "strangled",
    "brutality",
    "traffic collision",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [80]:
# Hard-coding cause_of_death for entry
df.loc[
    df[df["link"] == "https://en.wikipedia.org/wiki/Regis_Korchinski-Paquet"].index,
    "cause_of_death",
] = "possible police interference"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [81]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_1`

In [82]:
%%time

# Column to check
column = 'info_3_1'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 25137 values in cause_of_death column.

CPU times: total: 6.62 s
Wall time: 6.64 s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [83]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84569
2    12634
3      821
0        8
4        8
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_1`

In [84]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_1"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [85]:
# # Code to check each value
# value = roles_cause_list.pop()
# value

<IPython.core.display.Javascript object>

In [86]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_1"].notna()].index
#             if value in df.loc[index, "info_3_1"]
#         ],
#         "info_3_1",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [87]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [88]:
# # Example code to quick-check a specific entry
# df[df["info_3_1"] == value]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [89]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = []
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = []
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = []
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "Arts",
    "art",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = []
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = []
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = ["pilot"]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = []
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = []
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = []
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [90]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_1`

In [91]:
%%time

# Column to check
column = 'info_3_1'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 25137 values in cause_of_death column.

CPU times: total: 125 ms
Wall time: 97 ms


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [92]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84568
2    12634
3      822
0        8
4        8
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_1`

In [93]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_1"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [94]:
# # Code to check each value
# value = roles_cause_list.pop()
# value

<IPython.core.display.Javascript object>

In [95]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_1"].notna()].index
#             if value in df.loc[index, "info_3_1"]
#         ],
#         "info_3_1",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [96]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [97]:
# # Example code to quick-check a specific entry
# df[df["info_3_1"] == "agement at the Depment of Defense"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [98]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = ["Art"]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = []
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = []
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = ["society"]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = []
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = []
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = []
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = ["Man"]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = []
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = []
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [99]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_1`

In [100]:
%%time

# Column to check
column = 'info_3_1'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 25137 values in cause_of_death column.

CPU times: total: 93.8 ms
Wall time: 96 ms


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [101]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84568
2    12634
3      822
0        8
4        8
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Verifying that Values in info_3_1 Are Exhausted

In [102]:
# Verifying that `info_3_2` is exhausted
df["info_3_1"].value_counts()

    4664
Name: info_3_1, dtype: int64

<IPython.core.display.Javascript object>

#### Dropping `info_3_1`

In [103]:
# Dropping info_3_2
df.drop("info_3_1", axis=1, inplace=True)

# Checking sample
df.sample()

,day,name,info,link,num_references,year,month,info_parenth,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,info_3_2,info_4_0,info_4_1,info_4_2,info_5_0,info_5_1,info_5_2,info_6_0,info_6_1,info_7_0,info_8_0,info_8_1,info_9_0,info_10_0,info_11_0,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
57248,4,David M. Gates,", 94, American ecologist.",https://en.wikipedia.org/wiki/David_M._Gates,3,2016,March,None,94.0,None,United States of America,None,None,1.386294,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,0,0,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Observations:
- Our search of column `info_3_1` is finished and have dropped that column.
- We will now move on to searching `info_3_2`.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_2`

In [104]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [105]:
# # Code to check each value
# value = roles_cause_list.pop()
# value

<IPython.core.display.Javascript object>

In [106]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_2"].notna()].index
#             if value in df.loc[index, "info_3_2"]
#         ],
#         "info_3_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [107]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [108]:
# # Example code to quick-check a specific entry
# df[df["info_3_2"] == value]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [109]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "Lyndon B Johnson",
    "Retail Clerks Int'l",
    "Technology",
    "Immigration",
    "Bearsden",
    "arms development",
    "Social Services",
    "Bell Island",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "record producer",
    "appeared with the Chicago Symphony Orchestra",
    "author",
    "singer",
    "Stone the Crows",
    "the West",
    "producer",
    "executive",
    "screenwriter",
    "writer",
    "TV personality",
    "film",
    "television composer",
    "the CPB",
    "guitarist",
    "Dawn",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = ["the NHL New Devils", "yard dashes", "Sports Association of", "administrator"]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = ["Sciences", "Antarctic Administration"]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = ["Beaches Resorts"]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = ["language", "Space Museum", "professor emeritus", "lecturer"]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "ballistic missile programs",
    "II",
    "first husband of actress Marilyn Monroe",
    "Vice Minister of Public Security",
    "Marine aviator",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = ["the Grenadines"]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = ["plaintiff in their inheritance feud"]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = ["heart disease", "liver disease", "lymphoma"]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [110]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_2`

In [111]:
%%time

# Column to check
column = 'info_3_2'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 25138 values in cause_of_death column.

CPU times: total: 31.2 ms
Wall time: 30 ms


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [112]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84567
2    12632
3      825
0        8
4        8
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Verifying that Values in info_3_2 Are Exhausted

In [113]:
# Verifying that `info_3_2` is exhausted
df["info_3_2"].value_counts()

    46
Name: info_3_2, dtype: int64

<IPython.core.display.Javascript object>

#### Dropping info_3_2

In [114]:
# Dropping info_3_2
df.drop("info_3_2", axis=1, inplace=True)

# Checking sample
df.sample()

,day,name,info,link,num_references,year,month,info_parenth,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,info_4_0,info_4_1,info_4_2,info_5_0,info_5_1,info_5_2,info_6_0,info_6_1,info_7_0,info_8_0,info_8_1,info_9_0,info_10_0,info_11_0,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
35135,10,Frank Mascara,", 81, American politician, U.S. Representative from Pennsylvania , lung cancer.",https://en.wikipedia.org/wiki/Frank_Mascara,4,2011,July,,81.0,None,United States of America,United States of America,1995 2003,1.609438,lung cancer,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,1,0,0,0,1


<IPython.core.display.Javascript object>

#### Observations:
- Our search of column `info_3_2` is finished and have dropped that column.
- We will now move on to searching `info_4_0`.

#### Finding `known_for` Roles and `cause_of_death` in `info_4_0`

In [115]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_4_0"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [116]:
# # Code to check each value
# value = roles_cause_list.pop()
# value

<IPython.core.display.Javascript object>

In [117]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_4_0"].notna()].index
#             if value in df.loc[index, "info_4_0"]
#         ],
#         "info_4_0",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [118]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [119]:
# # Example code to quick-check a specific entry
# df[df["info_4_0"] == value]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [120]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = []
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = []
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = []
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = []
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = []
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = []
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "ordered KAL to be shot down",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = []
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = []
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "complications of radiation treatments for cancer",
    "cardiac arrest as a complication from cancer",
    "Complications arising from cancer",
    "multiple organ failure due to liver cancer",
    "complications linked to prostate cancer",
    "renal failure brought on by lung cancer",
    "esophageal cancer",
    "complications from pancreatic cancer",
    "complications from esophageal cancer",
    "complications from colorectal cancer",
    "prostate cancer complicated by COVID",
    "complications from carcinoid cancer",
    "complications of pancreatic cancer",
    "complications from prostate cancer",
    "complications from bladder cancer",
    "complications from ovarian cancer",
    "complications from stomach cancer",
    "complications from breast cancer",
    "complications from kidney cancer",
    "complications from throat cancer",
    "complications of prostate cancer",
    "complications from colon cancer",
    "pneumonia complicated by cancer",
    "complications from liver cancer",
    "complications of cancer surgery",
    "complications from lung cancer",
    "complications of colon cancer",
    "complications due to cancer",
    "cancer complicated by COVID",
    "complications from cancer",
    "endometrial serous cancer",
    "euthanized due to cancer",
    "complications of cancer",
    "neuroendocrine cancer",
    "salivary gland cancer",
    "cancer of the larynx",
    "biliary tract cancer",
    "gall bladder cancer",
    "bone marrow cancer",
    "oesophageal cancer",
    "gallbladder cancer",
    "endometrial cancer",
    "pancreatic cancer",
    "esophageal cancer",
    "colorectal cancer",
    "peritoneal cancer",
    "cancer of the jaw",
    "renal cell cancer",
    "intestinal cancer",
    "bile duct cancer",
    "lymphatic cancer",
    "laryngeal cancer",
    "carcinoid cancer",
    "abdominal cancer",
    "prostate cancer",
    "cervical cancer",
    "duodenal cancer",
    "stomach cancer",
    "ovarian cancer",
    "bladder cancer",
    "gastric cancer",
    "uterine cancer",
    "thyroid cancer",
    "breast cancer",
    "throat cancer",
    "kidney cancer",
    "tongue cancer",
    "spinal cancer",
    "thymus cancer",
    "brain cancer",
    "liver cancer",
    "colon cancer",
    "bowel cancer",
    "renal cancer",
    "blood cancer",
    "mouth cancer",
    "sinus cancer",
    "lung cancer",
    "bone cancer",
    "skin cancer",
    "neck cancer",
    "oral cancer",
    "jaw cancer",
    "eye cancer",
    "cancer",
    "head injuries sustained after a heart attack",
    "traffic collision following a heart attack",
    "heart attack as a complication of malaria",
    "complications following heart attack",
    "electric shock induced heart attack",
    "complications from a heart attack",
    "heart attack after drug overdose",
    "heart attack followed by stroke",
    "complications from heart attack",
    "complications of a heart attack",
    "heart attack due to cirrhosis",
    "heart attack during surgery",
    "drowned after heart attack",
    "suspected heart attack",
    "apparent heart attack",
    "heart attack",
    "complications from multiple organ failure caused by COVID",
    "meningoencephalitis complicated by COVID",
    "prostate cancer complicated by COVID",
    "ruptured brain aneurysm from COVID",
    "kidney complications from COVID",
    "pneumonia resulting from COVID",
    "leukemia complicated by COVID",
    "pulmonary embolism from COVID",
    "cancer complicated by COVID",
    "COVID induced septic shock",
    "cardiac arrest from COVID",
    "complications from COVID",
    "post COVID complications",
    "septic shock from COVID",
    "complications of COVID",
    "COVID related illness",
    "post COVID pneumonia",
    "COVID",
    "complications from heart failure",
    "congestive heart failure",
    "heart failure caused by",
    "suspected heart failure",
    "chronic heart failure",
    "heart failure",
    "renal failure brought on by lung cancer",
    "complications from lung cancer",
    "lung cancer",
    "complications from a series of strokes",
    "complications from multiple strokes",
    "complications from ischemic strokes",
    "complications from several strokes",
    "complications relating to a stroke",
    "complications related to a stroke",
    "complications of multiple strokes",
    "complications from basilar stroke",
    "complications following a stroke",
    "heart attack followed by stroke",
    "complications following stroke",
    "complications from a stroke",
    "complications from strokes",
    "following multiple strokes",
    "complications of a stroke",
    "complications from stroke",
    "Complications from stroke",
    "complications of stroke",
    "stroke related illness",
    "stroke complications",
    "hemorrhagic stroke",
    "multiple strokes",
    "heat stroke",
    "strokes",
    "stroke",
    "complications of pneumonia from leukemia",
    "multiple organ failure due to pneumonia",
    "pneumonia as a complication of COPD",
    "complications from bronchopneumonia",
    "pneumonia complicated by cancer",
    "pneumonia resulting from COVID",
    "complications from pneumonia",
    "complications of pneumonia",
    "leukemia induced pneumonia",
    "post COVID pneumonia",
    "bilateral pneumonia",
    "bronchopneumonia",
    '" pneumonia',
    "pneumonia",
    "cardiac arrest as a complication from cancer",
    "cardiac arrest following skiing accident",
    "cardiac arrest following spinal surgery",
    "cardiac arrest after heart ailment",
    "cardiac arrest from COVID",
    "cardiac arrest",
    "suspected suicide by gunshot",
    "found shot dead at his home",
    "apparent suicide by gunshot",
    "shot to death in Rotterdam",
    "assassination by gunshot",
    "multiple gunshot wounds",
    "homicide by gunshot",
    "suicide by gunshot",
    "suicide by shotgun",
    "gunshot wounds",
    "natural causes",
    "traffic collision following a heart attack",
    "injuries sustained in a traffic collision",
    "injuries sustained in traffic collision",
    "complications from a traffic collision",
    "complications from traffic collision",
    "injuries from a traffic collision",
    "injuries from traffic collision",
    "traffic collision",
    "complications from Alzheimer disease",
    "complications of Alzheimer disease",
    "Alzheimer disease",
    "after long illness",
    "complications from a bone marrow transplant to combat acute myeloid leukemia",
    "complications of pneumonia from leukemia",
    "complications from leukemia treatment",
    "leukemia complicated by COVID",
    "complications from leukemia",
    "leukemia induced pneumonia",
    "complications of leukemia",
    "acute myeloid leukemia",
    "myeloid leukemia",
    "leukemia",
    "multiple system atrophy as a complication of Parkinson disease",
    "complications from Parkinson disease",
    "complications of Parkinson disease",
    "Parkinson disease",
    "complications of heart disease",
    "heart disease",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [121]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_4_0`

In [122]:
%%time

# Column to check
column = 'info_4_0'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 30215 values in cause_of_death column.

CPU times: total: 11.7 s
Wall time: 11.7 s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [123]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84567
2    12632
3      825
0        8
4        8
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_4_0`

In [124]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_4_0"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [125]:
# # Code to check each value
# value = roles_cause_list.pop()
# value

<IPython.core.display.Javascript object>

In [126]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_4_0"].notna()].index
#             if value in df.loc[index, "info_4_0"]
#         ],
#         "info_4_0",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [127]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [128]:
# # Example code to quick-check a specific entry
# df[df["info_4_0"] == "oldest modern MP"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [129]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "and politician",
    "First Secretary of the SED fall of the Berlin Wall",
    "MP for Matale District",
    "MP for Belfast East",
    "MP for Brent North",
    "MP for Marlborough",
    "oldest modern MP",
    "MP for Cleveland",
    "MP for Luanshya",
    "MP for San José",
    "MP for Ainamoi",
    "MP for Leeds",
    "and MP",
    "MP",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "and author of more than books on chess",
    "and author",
    "and short story writer",
    "and singer songwriter",
    "and screenplay writer",
    "singer songwriter",
    "and travel writer",
    "and screenwriter",
    "and story writer",
    "and manga writer",
    "and food writer",
    "dialogue writer",
    "and songwriter",
    "Yiddish writer",
    "fiction writer",
    "script writer",
    "comedy writer",
    "screenwriter",
    "drama writer",
    "and writer",
    "songwriter",
    "writer",
    "composer of classical music",
    "and composer",
    "composer",
    "and author of more than books on chess",
    "co authored Christmas songs",
    "and science fiction author",
    "and comics author",
    "prolific author",
    "and poet",
    "poet",
    "director of original production of",
    "director of the National Theatre",
    "and documentary film director",
    "and film television director",
    "documentary film director",
    "and television director",
    "and artistic director",
    "and theatre director",
    "and theater director",
    "television director",
    "and opera director",
    "and stage director",
    "and film director",
    "cartoon director",
    "theater director",
    "stage director",
    "music director",
    "film director",
    "art director",
    "and journalist",
    "journalist",
    "and actor of Bollywood",
    "and television actor",
    "and Broadway actor;",
    "socialite & actor",
    "and voice actor",
    "and actor",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = []
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = []
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = []
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = []
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = []
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = []
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = []
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "shot",
    "complications from multiple organ failure",
    "euthanasia due to multiple organ failure",
    "multiple organ failure",
    "suicide by overdose of medication",
    "suicide by autodefenestration",
    "suspected suicide by hanging",
    "physician assisted suicide",
    "suicide by explosive vest",
    "suicide by exsanguination",
    "suicide by asphyxiation",
    "suicide under a train",
    "suspected suicide by",
    "suicide by explosion",
    "suicide by poisoning",
    "suicide by car crash",
    "suicide by drowning",
    "suicide by overdose",
    "suicide by hanging",
    "suicide by jumping",
    "suspected suicide",
    "assisted suicide",
    "suicide by train",
    "apparent suicide",
    "alleged suicide",
    "suicide",
    "kidney failure caused by diabetes",
    "kidney failure",
    '" complications from a fall',
    "complications from a fall",
    "complications from respiratory failure",
    "pulmonary respiratory failure",
    "acute respiratory failure",
    "respiratory failure",
    "head injury from a fall at a Rolling Stones concert",
    "complications from surgery after a fall",
    "blood clot as a result of a fall",
    "cerebral haemorrhage after fall",
    "brain injuries following a fall",
    "internal bleeding after a fall",
    "heart arrhythmia after a fall",
    "injuries sustained in a fall",
    "head injury following a fall",
    "broken neck following a fall",
    '" complications from a fall',
    "following a number of falls",
    "complications from a fall",
    "head injuries from a fall",
    "complications of a fall",
    "complications from fall",
    "brain injury from fall",
    "head injury from fall",
    "injuries from a fall",
    "illness after a fall",
    "mountaineering fall",
    "fall from tractor",
    "fall from balcony",
    "horseriding fall",
    "fall from a tree",
    "fall from Lhotse",
    "fall from ATV",
    "balcony fall",
    "ft fall",
    "complications of treatment for lymphoma",
    "complications of non Hodgkin lymphoma",
    "complications from lymphoma",
    "euthanized due to lymphoma",
    "large B cell lymphoma",
    "non Hodgkin lymphoma",
    "large cell lymphoma",
    "Hodgkin lymphoma",
    "Burkitt lymphoma",
    "lymphoma",
    "euthanized after contracting laminitis",
    "euthanized after race injury",
    "euthanized due to lymphoma",
    "euthanized by anaesthesia",
    "euthanized",
    "complications from a brain tumor",
    "brain tumor",
    "light plane crash",
    "aeroplane crash",
    "airplane crash",
    "plane crash",
    "complications from surgery after a fall",
    "complications from surgery",
    "complications from dementia",
    "illness related to dementia",
    "complications of dementia",
    "dementia with Lewy bodies",
    "frontotemporal dementia",
    "pugilistic dementia",
    "Lewy body dementia",
    "vascular dementia",
    "dementia",
    "complications from diabetes",
    "complications from melanoma",
    "metastatic melanoma",
    "ocular melanoma",
    "melanoma",
    "helicopter crash",
    "car accident in Moscow",
    "car accident",
    "complications from amyotrophic lateral sclerosis",
    "amyotrophic lateral sclerosis",
    "kidney failure caused by diabetes",
    "illnesses related to diabetes",
    "complications from diabetes",
    "complications of diabetes",
    "Complications of diabetes",
    "diabetes complications",
    "diabetes",
    "after short illness",
    "complications from kidney disease",
    "polycystic kidney disease",
    "kidney disease",
    "lung infection",
    "complications of pulmonary emphysema",
    "emphysema induced infection",
    "complications of emphysema",
    "pulmonary emphysema",
    "emphysema",
    "complications from multiple myeloma",
    "multiple myeloma",
    "drowned in a swimming pool",
    "drowned",
    "cerebral hemorrhage",
    "complications from lung disease",
    "lung disease",
    "crushed by tractor",
    "fall from tractor",
    "tractor accident",
    "acute renal failure",
    "renal failure",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [130]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_4_0`

In [131]:
%%time

# Column to check
column = 'info_4_0'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 31275 values in cause_of_death column.

CPU times: total: 21.5 s
Wall time: 21.8 s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [132]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84522
2    12634
3      867
4        9
0        8
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_4_0`

In [134]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_4_0"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [603]:
# # Code to check each value
# value = roles_cause_list.pop()
# value

<IPython.core.display.Javascript object>

In [604]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_4_0"].notna()].index
#             if value in df.loc[index, "info_4_0"]
#         ],
#         "info_4_0",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [605]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [607]:
# # Example code to quick-check a specific entry
# df[df["info_4_0"] == "Ambassador to s"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [608]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "member of the New South Legislative Assembly for Wagga Wagga",
    "founder of the New Sicily party",
    "co founder of the New Party",
    "Administrator of Papua New",
    "New Brunswick",
    "New Hampshire",
    "director of the Human Rights Foundation",
    "brother of playwright Larry Kramer",
    "and activist",
    "Illinois Board",
    "Illinois",
    "and diplomat",
    "diplomat",
    "member of the Ohio House of Representatives",
    "Ohio",
    "and judge",
    "Senator from Wisconsin",
    "Senator for City",
    "Senator",
    "outspoken critic of The Troubles",
    "anti communist politician",
    "and local politician",
    "politician",
    "Deputy Chief Minister",
    "Deputy Premier",
    "Deputy",
    "member of the North Carolina House of Representatives",
    "Ambassador to the Holy See and",
    "Ambassador to Micronesia",
    "Ambassador to and",
    "Ambassador to the",
    "Ambassador to s",
    "Ambassador to",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "author of instructional books on contract bridge",
    "and medical textbook author",
    "Chopin authority",
    "author of",
    "author",
    "founder of Theatre New Brunswick",
    "President of CBS News",
    "Newport Folk Festival",
    "Poet Laureate of New",
    "director of Folk Alliance International",
    "ex wife of actor Peter Lawford",
    "musical arranger for the Hi Los",
    "and arranger",
    "arranger",
    "and producer",
    "and playwright",
    "lead singer for The Ramones",
    "and Kunqu opera singer",
    "and folk singer",
    "and singer",
    "singer",
    "and novelist",
    "novelist",
    "and painter",
    "and producer of light music",
    "and theatrical producer",
    "and television producer",
    "and record producer",
    "and radio producer",
    "and film producer",
    "record producer",
    "video producer",
    "and producer",
    "producer",
    "editor in chief of magazine",
    "editor in chief of",
    "and editor of the",
    "newspaper editor",
    "and film editor",
    "editor of the",
    "and editor",
    "editor",
    "and playwright",
    "and essayist",
    "essayist",
    "and actress",
    "and photographer",
    "photographer",
    "and musician",
    "and artist",
    "and pianist",
    "pianist",
    "and publisher",
    "publisher",
    "and orchestra conductor",
    "and choral conductor",
    "and conductor",
    "conductor",
    "and television personality",
    "and a major figure in the advancement of crafts as a serious artistic discipline",
    "and installation artist",
    "and movie poster artist",
    "and performance artist",
    "and recording artist",
    "and ceramic artist",
    "and graphic artist",
    "and visual artist",
    "graphic artist",
    "visual artist",
    "video artist",
    "and artist",
    "artist",
    "principal dancer with the Royal Ballet",
    "and traditional dancer",
    "and dancer",
    "dancer",
    "Miss North Carolina",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "and athletic director",
    "coach at Arizona State University",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "flight director",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = ["Allstream Inc", "Inc", "and businessman"]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "and Warden of New College",
    "and museum director",
    "and historian of language",
    "military historian",
    "and science fiction historian",
    "and music historian",
    "and art historian",
    "chess historian",
    "and historian",
    "art historian",
    "historian",
    "and teacher",
    "and academic administrator",
    "and academic",
    "later served as University of Massachusetts President",
    "and music educator",
    "and sex educator",
    "music educator",
    "and educator",
    "educator",
    "and theatre teacher",
    "and acting teacher",
    "and vocal teacher",
    "and music teacher",
    "and art teacher",
    "and teacher",
    "teacher",
    "and translator",
    "translator",
    "Cambridge",
    "and professor of archaeology at University",
    "and professor at Tsinghua University",
    "and professor at Hebrew University",
    "and professor of literature",
    "and professor at MIT",
    "and professor",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "and military subcontractor",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = []
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = ["and director of Catholic Relief Services", "and philanthropist"]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = []
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "fall",
    "stabbed",
    "complications from pulmonary fibrosis",
    "idiopathic pulmonary fibrosis",
    "pulmonary fibrosis",
    "complications from heart surgery",
    "complications from liver failure",
    "acute liver failure",
    "liver failure",
    "complications from a brain tumour",
    "brain tumour",
    "complications from a brain hemorrhage",
    "brain hemorrhage",
    "homicide",
    "complications from motor neurone disease",
    "motor neurone disease",
    "injuries sustained in Brussels Airport bombings",
    "victim of Batasang Pambansa bombing",
    "injuries sustained in bombing",
    "bombing",
    "complications from pulmonary embolism",
    "pulmonary embolism",
    "sepsis complicated by a urinary tract infection",
    "complications from sepsis",
    "sepsis",
    "drone airstrike",
    "airstrike",
    "complications from heart bypass surgery",
    "complications from heart valve failure",
    "complications following heart surgery",
    "complications during heart surgery",
    "complications after heart surgery",
    "complications from heart surgery",
    "complications of heart surgery",
    "infection after heart surgery",
    "following open heart surgery",
    "heart surgery complications",
    "complications from heart",
    "heart related problems",
    "heart complications",
    "heart arrhythmia",
    "heart condition",
    "heart problems",
    "heart ailments",
    "heart ailment",
    "heart illness",
    "heart",
    "complications from a brain haemorrhage",
    "complications from brain haemorrhage",
    "brain haemorrhage following bout",
    "suspected brain haemorrhage",
    "brain haemorrhage",
    "complications from leukaemia",
    "acute myeloid leukaemia",
    "leukaemia",
    "complications from ruptured brain aneurysm",
    "complications of aortic aneurysm surgery",
    "abdominal aortic aneurysm",
    "ruptured aortic aneurysm",
    "ruptured aneurysm",
    "aortic aneurysm",
    "brain aneurysm",
    "aneurysm",
    "complications from liver disease",
    "complications of liver disease",
    "alcoholic liver disease",
    "liver disease",
    "execution by hanging",
    "mesothelioma",
    "complications of aortic aneurysm surgery",
    "abdominal aortic aneurysm",
    "ruptured aortic aneurysm",
    "aortic aneurysm",
    "complications from myelodysplastic syndrome",
    "myelodysplastic syndrome",
    "murdered his wife Annie Leong",
    "murdered by an assailant",
    "murdered by the FARC",
    "murdered",
    "complications from septic shock",
    "septic shock",
    "car crash while leading police on chase",
    "car crash",
    "glioblastoma",
    "accidental drowning",
    "drowning",
    "complications from COPD",
    "COPD",
    "complications from cirrhosis",
    "liver cirrhosis",
    "cirrhosis",
    "complications of organ failure",
    "organ failure",
    "AIDS related complications",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [609]:
# Dropping info_4_0 value for entries with extraneous value
df.loc[[index for index in df[df["info_4_0"] == "and"].index], "info_4_0"] = ""

# Dropping info_4_0 value for entries with redundant category value
df.loc[
    [
        index
        for index in df[df["info_4_0"].notna()].index
        if "Virginia" in df.loc[index, "info_4_0"]
    ],
    "info_4_0",
] = ""

# Dropping info_4_0 value for entries with extraneous value
df.loc[
    [
        index
        for index in df[
            df["link"] == "https://en.wikipedia.org/wiki/Rajeev_Motwani"
        ].index
    ],
    "info_4_0",
] = ""

# Dropping info_4_0 value for entries with redundant category value
df.loc[
    [
        index
        for index in df[df["info_4_0"].notna()].index
        if "Wisconsin" in df.loc[index, "info_4_0"]
    ],
    "info_4_0",
] = ""

# Dropping info_4_0 value for entries with extraneous value
df.loc[[index for index in df[df["info_4_0"] == "Oxford"].index], "info_4_0"] = ""

# Dropping info_4_0 value for entries with extraneous value
df.loc[[index for index in df[df["info_4_0"] == "Arizona"].index], "info_4_0"] = ""

# Dropping info_4_0 value for entries with extraneous value
df.loc[
    [index for index in df[df["info_4_0"] == "North Carolina"].index], "info_4_0"
] = ""

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [610]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_4_0`

In [611]:
%%time

# Column to check
column = 'info_4_0'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 31681 values in cause_of_death column.

CPU times: total: 12.9 s
Wall time: 12.9 s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [612]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84398
2    12708
3      916
4       10
0        8
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_4_0`

In [1099]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_4_0"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [1100]:
# # Code to check each value
# value = roles_cause_list.pop()
# value

<IPython.core.display.Javascript object>

In [1101]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_4_0"].notna()].index
#             if value in df.loc[index, "info_4_0"]
#             #             and df.loc[index, "sports"] == 1
#         ],
#         "info_4_0",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [1102]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [1103]:
# # Example code to quick-check a specific entry
# df[df["info_4_0"] == "fired football coach Woody Hayes"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [1104]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "Connecticut",
    "co founder of Reporters Committee for Freedom of the Press",
    "founder of the Kizito Mihigo Peace Foundation",
    "co founder of the Global Wind Energy Council",
    "co founder of Electronic Frontier Foundation",
    "co founder of Congressional Black Caucus",
    "co founder of the Social Liberal Party",
    "founder of the Moscow Helsinki Group",
    "founder of the Burning Man festival",
    "founder of the Popular Party",
    "founder of the Aryan Nations",
    "founder of Prison Fellowship",
    "co founder of the Democrats",
    "founder of the People Party",
    "founder of the Association",
    "and co founder of the SDP",
    "founder of Gun Owners of",
    "founder of Awami Tahreek",
    "co founder of the SDLP",
    "co founder of FoLAR",
    "founder of EDEK",
    "Oregon",
    "and feminist",
    "oldest surviving member of the House of Representatives",
    "member of the House of Representatives for Toledo West",
    "member of the House of Representatives for Tokyo",
    "member of the House of Representatives for",
    "member of the House of Representatives",
    "Dean of the Diplomatic Corps of Washington",
    "inventor of the term series of tubes",
    "and political activist",
    "and social activist",
    "deputy",
    "and Revisionist Zionist activist",
    "and anti apartheid activist",
    "and animal rights activist",
    "and civil rights activist",
    "and conservative activist",
    "and human rights activist",
    "and libertarian activist",
    "and gay rights activist",
    "and political activist",
    "environmental activist",
    "and social activist",
    "political activist",
    "and AIDS activist",
    "feminist activist",
    "peace activist",
    "socio political researcher",
    "Charter signatory",
    "involved in negotiations for the Alaska Native Claims Settlement Act",
    "and lawyer",
    "lawyer",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "and director",
    "director",
    "actor",
    "playwright",
    "and literary critic",
    "and cultural critic",
    "and theater critic",
    "and music critic",
    "literary critic",
    "culinary critic",
    "and film critic",
    "and art critic",
    "film critic",
    "and critic",
    "critic",
    "and television presenter",
    "and television actress",
    "pornographic actress",
    "; mother of actress",
    "actress",
    "co founder of the Notting Hill Carnival",
    "co founder of Teilifís na Gaeilge",
    "founder of magazine",
    "founder of Theatre",
    "founder of the University of the Madrigal Singers",
    "co founder of Rankin Bass Productions",
    "founder of the Shanghai Biennale",
    "a founder of modern literature",
    "founder of The Jargon Society",
    "and co founder of Swell Maps",
    "founder of Kalnirnay almanac",
    "founder of Philles Records",
    "co founder of Group Zero",
    "founder of Armet & Davis",
    "founder of Books on Tape",
    "founder of Kinotavr",
    "founder of Pax TV",
    "and folk musician",
    "jazz musician",
    "musician",
    "Oliver Twist",
    "painter",
    "lyricist of State Anthem of",
    "and lyricist",
    "lyricist",
    "and sculptor",
    "and adult filmmaker",
    "and filmmaker",
    "filmmaker",
    "and book illustrator",
    "and illustrator",
    "illustrator",
    "and literary critic",
    "and cartoonist",
    "and printmaker",
    "printmaker",
    "and dramatist",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "founder of the United Congress",
    "founder of International Gymnastics Hall of Fame",
    "and coach",
    "two time Olympic champion",
    "and Olympic champion",
    "Olympic champion",
    "and baseball coach",
    "head coach",
    "and coach",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "and philosopher of social science",
    "co inventor of Fuchs Sherr modulated neutron initiator",
    "claimed to be the inventor of Silly Putty",
    "inventor of the rainbow hologram",
    "inventor of gamma ray camera",
    "and inventor of electronic instruments",
    "and researcher during World War II",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "founder of Electronic Data Systems",
    "founder of the Norbrook Group",
    "co founder of March Engineering",
    "founder of B Dalton",
    "founder of CarMax",
    "inventor of the TV dinner",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "transhumanist philosopher",
    "co founder of Central Technological University",
    "founder of Hummel figurine museum",
    "founder of the Makah Museum",
    "co founder of BANU",
    "founder of GITAM",
    "co founder of the International Women Air & Space Museum",
    "and co founder of the Studio Museum in Harlem",
    "founder of Ricardo Brennand Institute",
    "founder of Strake Jesuit College Prep",
    "co founder of Notre Dame College",
    "founder of the Mori Art Museum",
    "emeritus professor at Scripps Institution of Oceanography",
    "professor at Divinity School",
    "and university professor",
    "professor of pediatrics",
    "literature professor",
    "University professor",
    "and dance professor",
    "and film professor",
    "and law professor",
    "and USC professor",
    "college professor",
    "professor",
    "and curator",
    "curator",
    "and arts activist",
    "specialist in Catholic history in Alaska",
    "restored the Minnesota Governor Residence",
    "academician",
    "academic",
    "fired football coach Woody Hayes",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "New South Police Commissioner",
    "founder of Liberation Front",
    "co founder of Hezbollah",
    "co founder of Royal Chicano Air Force",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "a founder of the Franciscan Friars of the Renewal",
    "co founder of Youth Specialties",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "founder of House on Capitol Hill",
    "co founder of Maggie Centres",
    "cofounder of the Kal",
    "founder of L'Arche",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = []
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "myocardial infarction",
    "complications from a liver transplant",
    "liver thrombosis",
    "liver infection",
    "liver illness",
    "liver ailment",
    "shooting",
    "asthma attack",
    "asthma",
    "accidental drug overdose",
    "suspected drug overdose",
    "drug overdose",
    "road accident",
    "injuries from a motorcycle accident",
    "motorcycle accident",
    "drone strike",
    "bomb blast",
    "house fire",
    "complications from cardiovascular disease",
    "cardiovascular disease",
    "illness relating to complications from an old injury",
    "complications from a long illness",
    "complications from an illness",
    "neurodegenerative illness",
    "after a long illness",
    "respiratory illness",
    "after brief illness",
    "lingering illness",
    "cardiac illness",
    "severe illness",
    "brief illness",
    "after illness",
    "liver illness",
    "long illness",
    "illness",
    "pulmonary edema",
    "complications from lung transplant",
    "complications from collapsed lung",
    "complications of collapsed lung",
    "complications from lung surgery",
    "small cell lung carcinoma",
    "complications from lung",
    "lung complications",
    "blood clot in lung",
    "lung congestion",
    "lung problems",
    "lung failure",
    "lung ailment",
    "lung injury",
    "lung",
    "complications from aortic dissection",
    "aortic dissection",
    "cardiopulmonary arrest",
    "animal euthanasia",
    "euthanasia",
    "air strike",
    "euthanised following paddock accident",
    "euthanised following stomach tumour",
    "euthanised",
    "chest infection",
    "traffic accident",
    "complications from hypertension",
    "hypertension",
    "car bomb",
    "respiratory problems",
    "surgical complications",
    "Pick disease",
    "respiratory infection",
    "assassinated",
    "chronic obstructive pulmonary disease",
    "avalanche",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [1105]:
# Dropping info_4_0 value for entries with duplicate category value
df.loc[df[df["info_4_0"] == "New"].index, "info_4_0"] = ""

# Dropping info_4_0 value for entries with duplicate category value
df.loc[df[df["info_4_0"] == "Massachusetts"].index, "info_4_0"] = ""

# Dropping info_4_0 value for entries with duplicate category value
df.loc[df[df["info_4_0"] == "Jr"].index, "info_4_0"] = ""

# Dropping info_4_0 value for entries with duplicate category value
df.loc[df[df["info_4_0"] == "Washington"].index, "info_4_0"] = ""

# Dropping info_4_0 value for entries with duplicate category value
df.loc[df[df["info_4_0"] == "Michigan"].index, "info_4_0"] = ""

# Hard-coding info_4_0 value for entry to correctly categorize
df.loc[
    df[df["link"] == "https://en.wikipedia.org/wiki/H._Fred_Clark"].index, "info_4_0"
] = "humanitarian"

# Dropping info_4_0 value for entries with duplicate category value
df.loc[df[df["info_4_0"] == "Alaska"].index, "info_4_0"] = ""

# Dropping info_4_0 value for entries with duplicate category value
df.loc[df[df["info_4_0"] == "Minnesota"].index, "info_4_0"] = ""

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [1106]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_4_0`

In [1107]:
%%time

# Column to check
column = 'info_4_0'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 31856 values in cause_of_death column.

CPU times: total: 12.9 s
Wall time: 13 s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [1108]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    84354
2    12718
3      950
4       10
0        8
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

In [1109]:
print("dunzo!")

# Sound notification when cell executes
chime.success()

dunzo!


<IPython.core.display.Javascript object>

#### Finding `known_for` Roles and `cause_of_death` in `info_4_0`

In [ ]:
# Obtaining values for column and their counts
roles_cause_list = df["info_4_0"].value_counts(ascending=True).index.tolist()

In [ ]:
# Code to check each value
value = roles_cause_list.pop()
value

In [ ]:
# Create specific_roles_cause_list for above popped value
specific_roles_cause_list = (
    df.loc[
        [
            index
            for index in df[df["info_4_0"].notna()].index
            if value in df.loc[index, "info_4_0"]
        ],
        "info_4_0",
    ]
    .value_counts()
    .index.tolist()
)

In [ ]:
# Viewing list sorted by descending length to copy to dictionary below and screen values
sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

In [ ]:
# Example code to quick-check a specific entry
df[df["info_4_0"] == value]

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [ ]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = []
politics_govt_law = sorted(list(set(politics_govt_law)), key=lambda x: len(x), reverse=True)  

arts = []
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)  

sports = []
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True) 

sciences = []
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True) 

business_farming = []
business_farming = sorted(list(set(business_farming)), key=lambda x: len(x), reverse=True)  

academia_humanities = []
academia_humanities = sorted(list(set(academia_humanities)), key=lambda x: len(x), reverse=True)  

law_enf_military_operator = []
law_enf_military_operator = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)  

spiritual = []
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)  

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)  

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)  

event_record_other = []
event_record_other = sorted(list(set(event_record_other)), key=lambda x: len(x), reverse=True)  

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True) 

cause_of_death = []
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [ ]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,

}

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_4_0`

In [ ]:
%%time

# Column to check
column = 'info_4_0'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                if df.loc[index, 'cause_of_death']:
                    df.loc[index, 'cause_of_death'] = df.loc[index, 'cause_of_death'] + '/' + cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                else:
                    df.loc[index, 'cause_of_death'] = cause
                    df.loc[index, column] = item.replace(cause, '').strip()
                
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

#### Checking Updated `num_categories` Value Counts

In [ ]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Verifying that Values in info_3_0 Are Exhausted

In [ ]:
# # Verifying that `info_3_0` is exhausted
# df["info_3_0"].value_counts()

#### Dropping info_3_0

In [ ]:
# # Dropping info_3_0
# df.drop("info_3_0", axis=1, inplace=True)

# # Checking sample
# df.sample()

#### Observations:
- Our search of column `info_3_0` is finished and have dropped that column.
- We will now save our dataset and pick back up in a new notebook.

### Exporting Dataset to SQLite Database [wp_life_expect_clean7.db]()

In [ ]:
# # Exporting dataframe

# # Saving dataset in a SQLite database
# conn = sql.connect("wp_life_expect_clean7.db")
# df.to_sql("wp_life_expect_clean7", conn, index=False)

# # Chime notification when cell executes
# chime.success()

# [Proceed to Data Cleaning Part ]()